# 01 Scraping_Playwright

### Ziel: Scraping der Trump-Daten 
- Website: https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc 
- mithilfe von Playwright 
- im Folgenden findet sich meine Historie, wie ich zu einem funktionierenden Code mit Playwright kam
- plus ein Fail mit Selectolax
- PS: Falls Sie lieber sofort den funktionierenden Code sehen wollen, können Sie bis zum Ende des Notebooks scrollen.

## Playwright

Zuerst Folgendes installieren:

In [2]:
pip install playwright pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [playwright]3 [playwright]
Note: you may need to restart the kernel to use updated packages.


In [1]:
#playwright install 
#in shell ausführen

### Wie sollen die Daten am Ende aussehen?
- Id: Nummer des Posts
- author: Donald Trump @realdonaldtrump
- date_full: ganzes Datum
- day
- month
- year
- platform: Truthsocial or X (Twitter)
- text
- video/screenshot mit Transkription?

### Wie sieht die Website aus:

- Quellcode auf Website anschauen: https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc
- Seite lädt Inhalte interaktiv mit Java-Script nach (nicht statisch)
- alle Posts sind in jeweils einzelnen Blöcken gespeichert
- Suchmaske auf der Website implementiert
- tägliche Ergänzung neuer Posts
- Blick auf die URL: Beim Scrollen ändert sich die Seitenzahl in der URL
- zum 1.August waren es 87.640 Posts (X und Truthsocial)
- wahrscheinlich circa 3.000 Seiten

### Wahl des Tools:
- Beautifulsoup: schon älter, nur für statische Websites geeignet, braucht länger
- Selenium: dynamische Alternative, gute Möglichkeit
- Selectolax: modern und deutlich schneller als Beautiful, wird außerdem seltener blockiert, allerdings ebenfalls nur für statische Seiten
- Playwright: relativ modern, sehr schnell und effizient, für dynamische Seiten geeignet

### Wie viele Posts gibt es überhaupt?
- Auf der Webseite steht, dass es 87.656 gibt (Stand 6.8.2025)
- Aber gefunden werden irgendwie 126.778 Posts?!?

In [ ]:
# lädt langsam und ab 1000 Posts nicht mehr neu :(
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "trump_playwright_final.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    os.makedirs("images", exist_ok=True)  # einmalig vor der Schleife
    while True:
        item = await queue.get()
        if item is None:  # Stop-Signal
            queue.task_done()
            break

        image_url, post, filename = item #statt post_idx
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    #with open(fpath, "wb") as f:
                        #f.write(await resp.read())
                    #posts_data[post_idx]["image_path"] = fpath
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

            
def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Content-Zeilen finden
        #if len(lines) > 2 and lines[2].startswith("View"): # statt "View on"
        #    content_lines = lines[3:]
        #else:
        #    content_lines = lines[2:]
        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Text sauber zusammensetzen (mit Absatz-Trennung)
    content_text = "\n".join(content_lines).strip()

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text # statt "\n".join(content_lines).strip()
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text #"\n".join(content_lines).strip()
        }
# -------- Duplikat-Key --------
def make_post_key(data, include_image=True, include_text=True):
    """
    Generiert einen eindeutigen Schlüssel für einen Post.
    - Text optional
    - Bild optional
    - Nur Posts ohne jegliche Metadaten UND ohne Text UND ohne Bild werden verworfen
    """

    # Metadaten absichern (NaN -> "", None -> "")
    author = str(data.get("author", "") or "").strip()
    platform = str(data.get("platform", "") or "").strip()
    date = str(data.get("date", "") or "").strip()
    time = str(data.get("time", "") or "").strip()
    text = str(data.get("text", "") or "").strip()
    img = data.get("image_path")

    # Wenn absolut nichts da → kein valider Key
    if not (author or platform or date or text or img):
        return None

    parts = []
    parts.extend([author, platform, date, time])

    # Text einbauen (optional)
    if include_text and text:
        # normalize whitespace → verhindert Unterschiede nur wegen \n oder Mehrfach-Leerzeichen
        text_norm = re.sub(r"\s+", " ", text)
        parts.append(text_norm)

    # Bild einbauen (optional)
    if include_image and isinstance(img, str) and img.strip():
        parts.append(os.path.basename(img.strip()))

    # Key zusammenbauen
    raw_key = "|".join(parts).strip()
    if not raw_key:
        return None

    return hashlib.md5(raw_key.encode("utf-8")).hexdigest()


async def scrape_all_dynamic(max_posts=900, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzung ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE).fillna("") #fillna neu
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data if make_post_key(row)} #ab if neu
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")
    else:
        posts_data = []
        seen_posts = set()

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    break

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None,
                        "image_url": None #neu
                    }

                    # Bild-Download vorbereiten
                    img_src = None #neu
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                post["image_url"] = src #neu
                                img_src = src #neu
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                post["image_path"] = filename #neu
                                await queue.put((src, post, filename))
                    except Exception as e:
                        print(f"Bildfehler: {e}")

                    post["image_url"] = img_src #neu
                    
                    # Schlüssel generieren
                    key = make_post_key(post, include_text=True, include_image=True)
                    if not key or key in seen_posts:
                        continue
                    seen_posts.add(key)
                    
                    posts_data.append(post)
                    new_count += 1

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            # Scrollen und no_new_rounds prüfen
            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen nur, wenn max_posts noch nicht erreicht
            if len(posts_data) < max_posts:
                last_height = await page.evaluate("document.body.scrollHeight")
                await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
                await asyncio.sleep(3)
                new_height = await page.evaluate("document.body.scrollHeight")
                if new_height == last_height:
                    print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                    break
            else:
                print(f"Maximale Anzahl {max_posts} erreicht.")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    fff = pd.DataFrame(posts_data)
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")


# --- Analyse: Wie viele Posts enthalten Text ---
    total = len(fff)
    no_text = fff["text"].isna().sum()
    with_text = total - no_text
    print("===================================")
    print(f"Gesamt:     {total}")
    print(f"Ohne Text:  {no_text} ({no_text/total:.1%})")
    print(f"Mit Text:   {with_text} ({with_text/total:.1%})")
    print("===================================")

# Starten
await scrape_all_dynamic(max_posts=90000)

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "trump_playwright_final.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    os.makedirs("images", exist_ok=True)  # einmalig vor der Schleife
    while True:
        item = await queue.get()
        if item is None:  # Stop-Signal
            queue.task_done()
            break

        image_url, post, filename = item #statt post_idx
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    #with open(fpath, "wb") as f:
                        #f.write(await resp.read())
                    #posts_data[post_idx]["image_path"] = fpath
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

            
def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Content-Zeilen finden
        #if len(lines) > 2 and lines[2].startswith("View"): # statt "View on"
        #    content_lines = lines[3:]
        #else:
        #    content_lines = lines[2:]
        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Text sauber zusammensetzen (mit Absatz-Trennung)
    content_text = "\n".join(content_lines).strip()

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text # statt "\n".join(content_lines).strip()
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text #"\n".join(content_lines).strip()
        }
# -------- Duplikat-Key --------
def make_post_key(data, include_image=True, include_text=True):
    """
    Generiert einen eindeutigen Schlüssel für einen Post.
    - Text optional
    - Bild optional
    - Nur Posts ohne jegliche Metadaten UND ohne Text UND ohne Bild werden verworfen
    """
    # --- Gültigkeit prüfen ---
    # -> wenn keine Metadaten vorhanden, kein Key
    # Metadaten prüfen
    author = str(data.get("author", "")).strip()
    platform = str(data.get("platform", "")).strip()
    date = str(data.get("date", "")).strip()
    time = str(data.get("time", "")).strip()
    text = str(data.get("text", "")).strip()
    img = data.get("image_path")
    
     # Wenn gar nichts da ist → kein valider Key
    if not (author or platform or date or text or img):
        return None
    
    parts = []
    # Basis-Metadaten immer einbauen
    parts.extend([author, platform, date, time])
    
    ## Author, Date, Time # in schneller
    #for field in ["author", "date", "time"]:
    #    val = str(data.get(field, "")).strip().lower()
    #    parts.append(val)

    # Handle/Author
    #author_val = data.get("author", "")
    #if pd.isna(author_val):
    #    author_val = ""
    #parts.append(str(author_val).strip().lower())

    # Datum
    #date_val = data.get("date", "")
    #if pd.isna(date_val):
    #    date_val = ""
    #parts.append(str(date_val).strip())

    # Zeit
    #time_val = data.get("time", "")
    #if pd.isna(time_val):
    #    time_val = ""
    #parts.append(str(time_val).strip())

    # Text (optional)
    #if include_text:
    #    text_val = str(data.get("text", "")).strip().lower()
    #    text_norm = re.sub(r"\s+", " ", text_val)
    #    parts.append(text_norm)

    # Bild (optional)
    #if include_image:
    #    img_url = data.get("image_url")
    #    if img_url:
            # Hash statt kompletter URL
    #        img_hash = hashlib.md5(img_url.encode("utf-8")).hexdigest()
    #        parts.append(img_hash)
    #    else:
    #        parts.append("")


    # Key zusammenbauen
    #raw_key = "|".join(parts)
    #return raw_key #hashlib.sha256(raw_key.encode("utf-8")).hexdigest()
    
    # Text optional
    #if include_text:
    #    text = str(post.get("text", "")).strip()
        # nur hinzufügen, wenn sinnvoller Text vorhanden
    #    if len(text) > 5:  
    #        parts.append(text)

    # Bild optional
    #if include_image:
    #    img = post.get("image_path")
    #    if img:
    #        parts.append(os.path.basename(img))
    
    # Text optional
    if include_text and text:
        parts.append(text)

    # Bild optional
    if include_image and img:
        parts.append(os.path.basename(img))


    raw_key = "|".join(parts).strip()
    if not raw_key:
        return None

    return hashlib.md5(raw_key.encode("utf-8")).hexdigest()



async def scrape_all_dynamic(max_posts=900, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzung ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE)
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data}
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    break

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None,
                        "image_url": None #neu
                    }

                    # Bild-Download vorbereiten
                    img_src = None #neu
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                img_src = src #neu
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                await queue.put((src, post, filename))
                    except:
                        pass

                    post["image_url"] = img_src #neu
                    
                    # Schlüssel generieren
                    key = make_post_key(post, include_text=True, include_image=True)
                    if not key or key in seen_posts:
                        continue
                    seen_posts.add(key)
                    
                    posts_data.append(post)
                    new_count += 1

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            # Scrollen und no_new_rounds prüfen
            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen nur, wenn max_posts noch nicht erreicht
            if len(posts_data) < max_posts:
                last_height = await page.evaluate("document.body.scrollHeight")
                await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
                await asyncio.sleep(3)
                new_height = await page.evaluate("document.body.scrollHeight")
                if new_height == last_height:
                    print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                    break
            else:
                print(f"Maximale Anzahl {max_posts} erreicht.")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    fff = pd.DataFrame(posts_data)
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")


# --- Analyse: Wie viele Posts enthalten Text ---
    total = len(fff)
    no_text = fff["text"].isna().sum()
    with_text = total - no_text
    print("===================================")
    print(f"Gesamt:     {total}")
    print(f"Ohne Text:  {no_text} ({no_text/total:.1%})")
    print(f"Mit Text:   {with_text} ({with_text/total:.1%})")
    print("===================================")

# Starten
await scrape_all_dynamic(max_posts=90000)

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl

nest_asyncio.apply()  # Für Jupyter

os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()
        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def scrape_all_dynamic(max_posts=90000):
    posts_data = []
    seen_posts = set()
    no_new_rounds = 0
    max_no_new = 10  # Stop, wenn 5x hintereinander keine neuen Posts

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")

        while True:
            await page.wait_for_selector("div.block", timeout=150000)
            blocks = await page.query_selector_all("div.block")
            print(f"Aktuell {len(blocks)} Posts gespeichert")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    print(f"Ziel von {max_posts} Posts erreicht.")
                    await browser.close()
                    df = pd.DataFrame(posts_data)
                    df.to_csv("factbase_posts_playwright_dynamic.csv", index=False, encoding="utf-8")
                    print(f"{len(df)} Posts gespeichert.")
                    return

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    key = f"{data['author']}|{data['date']}|{data['time']}|{data['text']}"
                    if key in seen_posts:
                        continue
                    seen_posts.add(key)

                    # Bild finden
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path
                    })
                    new_count += 1
                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds} mal hintereinander)")
                if no_new_rounds >= max_no_new:
                    print("Keine neuen Inhalte mehr – Stop.")
                    break
            else:
                no_new_rounds = 0

            # Scrollen zum Nachladen
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(2)

        await browser.close()

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright_dynamic1.csv", index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(df)} Posts.")

# Für Notebook:
await scrape_all_dynamic(max_posts=90000)

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl

sslcontext = ssl.create_default_context(cafile=certifi.where())

nest_asyncio.apply()  # Für Jupyter-Umgebungen

# Ordner für Bilder anlegen, falls nicht da
os.makedirs("images", exist_ok=True)


sslcontext = ssl.create_default_context()
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None


def extract_metadata_text(text: str):
    # ... (Deine bestehende Funktion, unverändert)
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()

        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 8000

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Bild-URL suchen
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    # Bild herunterladen und lokal speichern
                    local_image_path = None
                    if image_url:
                        # Dateiname aus URL extrahieren oder generieren (z.B. Post-Datum + Zeit + zufällige Nummer)
                        filename = image_url.split("/")[-1].split("?")[0]
                        # Optional: Präfix mit Datum oder Post-Zeit zum besseren Sortieren
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    # Liste umkehren, damit ältester Post id=1 bekommt
    posts_data.reverse()

    # IDs vergeben
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright11.csv", index=False, encoding="utf-8")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright11.csv'.")

# Für Notebook:
await scrape_factbase()

In [20]:
#### Dauer dieses Codes: ca. 2,5h
# mit requests und selectolax 
#Folgendes stammt aus meinem ersten Versuch mit Selectolax
import requests

def count_posts():
    base_url = "https://rollcall.com/wp-json/wp/v2/posts"
    params = {
        "post_type": "factbase_tweet",
        "page": 1,
        "per_page": 100  # max 100 erlaubt
    }

    total_posts = 0
    while True:
        print(f"Lade Seite {params['page']}...")
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if not data:
                break
            total_posts += len(data)
            params["page"] += 1
        else:
            print(f"Fehler bei Seite {params['page']}: {response.status_code}")
            break

    print(f"Gesamtzahl der Posts: {total_posts}")

count_posts()


⏳ Lade Seite 1...
⏳ Lade Seite 2...
⏳ Lade Seite 3...
⏳ Lade Seite 4...
⏳ Lade Seite 5...
⏳ Lade Seite 6...
⏳ Lade Seite 7...
⏳ Lade Seite 8...
⏳ Lade Seite 9...
⏳ Lade Seite 10...
⏳ Lade Seite 11...
⏳ Lade Seite 12...
⏳ Lade Seite 13...
⏳ Lade Seite 14...
⏳ Lade Seite 15...
⏳ Lade Seite 16...
⏳ Lade Seite 17...
⏳ Lade Seite 18...
⏳ Lade Seite 19...
⏳ Lade Seite 20...
⏳ Lade Seite 21...
⏳ Lade Seite 22...
⏳ Lade Seite 23...
⏳ Lade Seite 24...
⏳ Lade Seite 25...
⏳ Lade Seite 26...
⏳ Lade Seite 27...
⏳ Lade Seite 28...
⏳ Lade Seite 29...
⏳ Lade Seite 30...
⏳ Lade Seite 31...
⏳ Lade Seite 32...
⏳ Lade Seite 33...
⏳ Lade Seite 34...
⏳ Lade Seite 35...
⏳ Lade Seite 36...
⏳ Lade Seite 37...
⏳ Lade Seite 38...
⏳ Lade Seite 39...
⏳ Lade Seite 40...
⏳ Lade Seite 41...
⏳ Lade Seite 42...
⏳ Lade Seite 43...
⏳ Lade Seite 44...
⏳ Lade Seite 45...
⏳ Lade Seite 46...
⏳ Lade Seite 47...
⏳ Lade Seite 48...
⏳ Lade Seite 49...
⏳ Lade Seite 50...
⏳ Lade Seite 51...
⏳ Lade Seite 52...
⏳ Lade Seite 53...
⏳ 

⏳ Lade Seite 417...
⏳ Lade Seite 418...
⏳ Lade Seite 419...
⏳ Lade Seite 420...
⏳ Lade Seite 421...
⏳ Lade Seite 422...
⏳ Lade Seite 423...
⏳ Lade Seite 424...
⏳ Lade Seite 425...
⏳ Lade Seite 426...
⏳ Lade Seite 427...
⏳ Lade Seite 428...
⏳ Lade Seite 429...
⏳ Lade Seite 430...
⏳ Lade Seite 431...
⏳ Lade Seite 432...
⏳ Lade Seite 433...
⏳ Lade Seite 434...
⏳ Lade Seite 435...
⏳ Lade Seite 436...
⏳ Lade Seite 437...
⏳ Lade Seite 438...
⏳ Lade Seite 439...
⏳ Lade Seite 440...
⏳ Lade Seite 441...
⏳ Lade Seite 442...
⏳ Lade Seite 443...
⏳ Lade Seite 444...
⏳ Lade Seite 445...
⏳ Lade Seite 446...
⏳ Lade Seite 447...
⏳ Lade Seite 448...
⏳ Lade Seite 449...
⏳ Lade Seite 450...
⏳ Lade Seite 451...
⏳ Lade Seite 452...
⏳ Lade Seite 453...
⏳ Lade Seite 454...
⏳ Lade Seite 455...
⏳ Lade Seite 456...
⏳ Lade Seite 457...
⏳ Lade Seite 458...
⏳ Lade Seite 459...
⏳ Lade Seite 460...
⏳ Lade Seite 461...
⏳ Lade Seite 462...
⏳ Lade Seite 463...
⏳ Lade Seite 464...
⏳ Lade Seite 465...
⏳ Lade Seite 466...


⏳ Lade Seite 827...
⏳ Lade Seite 828...
⏳ Lade Seite 829...
⏳ Lade Seite 830...
⏳ Lade Seite 831...
⏳ Lade Seite 832...
⏳ Lade Seite 833...
⏳ Lade Seite 834...
⏳ Lade Seite 835...
⏳ Lade Seite 836...
⏳ Lade Seite 837...
⏳ Lade Seite 838...
⏳ Lade Seite 839...
⏳ Lade Seite 840...
⏳ Lade Seite 841...
⏳ Lade Seite 842...
⏳ Lade Seite 843...
⏳ Lade Seite 844...
⏳ Lade Seite 845...
⏳ Lade Seite 846...
⏳ Lade Seite 847...
⏳ Lade Seite 848...
⏳ Lade Seite 849...
⏳ Lade Seite 850...
⏳ Lade Seite 851...
⏳ Lade Seite 852...
⏳ Lade Seite 853...
⏳ Lade Seite 854...
⏳ Lade Seite 855...
⏳ Lade Seite 856...
⏳ Lade Seite 857...
⏳ Lade Seite 858...
⏳ Lade Seite 859...
⏳ Lade Seite 860...
⏳ Lade Seite 861...
⏳ Lade Seite 862...
⏳ Lade Seite 863...
⏳ Lade Seite 864...
⏳ Lade Seite 865...
⏳ Lade Seite 866...
⏳ Lade Seite 867...
⏳ Lade Seite 868...
⏳ Lade Seite 869...
⏳ Lade Seite 870...
⏳ Lade Seite 871...
⏳ Lade Seite 872...
⏳ Lade Seite 873...
⏳ Lade Seite 874...
⏳ Lade Seite 875...
⏳ Lade Seite 876...


⏳ Lade Seite 1226...
⏳ Lade Seite 1227...
⏳ Lade Seite 1228...
⏳ Lade Seite 1229...
⏳ Lade Seite 1230...
⏳ Lade Seite 1231...
⏳ Lade Seite 1232...
⏳ Lade Seite 1233...
⏳ Lade Seite 1234...
⏳ Lade Seite 1235...
⏳ Lade Seite 1236...
⏳ Lade Seite 1237...
⏳ Lade Seite 1238...
⏳ Lade Seite 1239...
⏳ Lade Seite 1240...
⏳ Lade Seite 1241...
⏳ Lade Seite 1242...
⏳ Lade Seite 1243...
⏳ Lade Seite 1244...
⏳ Lade Seite 1245...
⏳ Lade Seite 1246...
⏳ Lade Seite 1247...
⏳ Lade Seite 1248...
⏳ Lade Seite 1249...
⏳ Lade Seite 1250...
⏳ Lade Seite 1251...
⏳ Lade Seite 1252...
⏳ Lade Seite 1253...
⏳ Lade Seite 1254...
⏳ Lade Seite 1255...
⏳ Lade Seite 1256...
⏳ Lade Seite 1257...
⏳ Lade Seite 1258...
⏳ Lade Seite 1259...
⏳ Lade Seite 1260...
⏳ Lade Seite 1261...
⏳ Lade Seite 1262...
⏳ Lade Seite 1263...
⏳ Lade Seite 1264...
⏳ Lade Seite 1265...
⏳ Lade Seite 1266...
⏳ Lade Seite 1267...
⏳ Lade Seite 1268...
⏳ Lade Seite 1269...
❌ Fehler bei Seite 1269: 400
✅ Gesamtzahl der Posts: 126778


### Versuch Nummer 1:
- Scraping klappt!
- das Videoformat wird bei jedem einzelnen Post mit "Ja" beantwortet und das Transkript der Videos (ALT-Text für alternativ) ist auch immer identisch
- die URL des Sceenshots sind ebenfalls nicht funktionstüchtig
- im Textblock sind noch weitere Informationen wie Datum, Plattform und Autor enthalten (diese sollten in den nächsten Schritten getrennt werden)

In [3]:
#### Dauer bis zum Ergebnis: ca. 3h
import asyncio 
#asyncio: await und async helfen dabei, nicht blockiert zu werden und das Nachladen der Seiten abzuwarten
import nest_asyncio
import pandas as pd
from playwright.async_api import async_playwright

nest_asyncio.apply()  # notwendig in Jupyter Notebooks

def is_probable_video(image_url: str, alt_text: str) -> bool:
    """Einfache Videoerkennung basierend auf URL und ALT-Text."""
    if not image_url:
        return False
    return (
        "video" in image_url.lower()
        or ".mp4" in image_url.lower()
        or "watch" in image_url.lower()
        or "play" in alt_text.lower()
        or "media from" in alt_text.lower()
    )

async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 3000  # als Limit gegen Endlosschleifen

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(600000) 
        #timeout muss hoch genug sein, damit die nachladenden Seiten aufgefangen werden

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping Seite {page_num}...")
            
            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=10000)
            except Exception as e:
                print(f"Seite {page_num} konnte nicht geladen werden: {e}")
                break

            blocks = await page.query_selector_all("div.block")
            # Website speichert die einzelnen Posts in einzelnen Blöcken dafür.

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    # kompletten sichtbaren Textinhalt des Blocks extrahieren
                    full_text = await block.inner_text()

                    # Screenshot-Information
                    image_el = await block.query_selector("img")
                    image_url = await image_el.get_attribute("src") if image_el else ""
                    alt_text = await image_el.get_attribute("alt") if image_el else ""

                    has_video = is_probable_video(image_url, alt_text)

                    posts_data.append({
                        "Text": full_text.strip(),
                        "Screenshot_URL": image_url.strip() if image_url else "",
                        "ALT_Text": alt_text.strip() if alt_text else "",
                        "Video_erkannt": "Ja" if has_video else "Nein"
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    # Speichern als CSV
    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright1.csv", index=False, encoding="utf-8-sig")
    print(f"\n {len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright1.csv'.")

# In Jupyter Notebook:
await scrape_factbase()

# In einem .py-Skript stattdessen:
# asyncio.run(scrape_factbase())


🔍 Scraping Seite 1...
🔍 Scraping Seite 2...
🔍 Scraping Seite 3...
🔍 Scraping Seite 4...
🔍 Scraping Seite 5...
🔍 Scraping Seite 6...
🔍 Scraping Seite 7...
🔍 Scraping Seite 8...
🔍 Scraping Seite 9...
🔍 Scraping Seite 10...
🔍 Scraping Seite 11...
🔍 Scraping Seite 12...
🔍 Scraping Seite 13...
🔍 Scraping Seite 14...
🔍 Scraping Seite 15...
🔍 Scraping Seite 16...
🔍 Scraping Seite 17...
🔍 Scraping Seite 18...
🔍 Scraping Seite 19...
🔍 Scraping Seite 20...
🔍 Scraping Seite 21...
🔍 Scraping Seite 22...
🔍 Scraping Seite 23...
🔍 Scraping Seite 24...
🔍 Scraping Seite 25...
🔍 Scraping Seite 26...
🔍 Scraping Seite 27...
🔍 Scraping Seite 28...
🔍 Scraping Seite 29...
🔍 Scraping Seite 30...
🔍 Scraping Seite 31...
🔍 Scraping Seite 32...
🔍 Scraping Seite 33...
🔍 Scraping Seite 34...
🔍 Scraping Seite 35...
🔍 Scraping Seite 36...
🔍 Scraping Seite 37...
🔍 Scraping Seite 38...
🔍 Scraping Seite 39...
🔍 Scraping Seite 40...
🔍 Scraping Seite 41...
🔍 Scraping Seite 42...
🔍 Scraping Seite 43...
🔍 Scraping Seite 44.

🔍 Scraping Seite 334...
🔍 Scraping Seite 335...
🔍 Scraping Seite 336...
🔍 Scraping Seite 337...
🔍 Scraping Seite 338...
🔍 Scraping Seite 339...
🔍 Scraping Seite 340...
🔍 Scraping Seite 341...
🔍 Scraping Seite 342...
🔍 Scraping Seite 343...
🔍 Scraping Seite 344...
🔍 Scraping Seite 345...
🔍 Scraping Seite 346...
🔍 Scraping Seite 347...
🔍 Scraping Seite 348...
🔍 Scraping Seite 349...
🔍 Scraping Seite 350...
🔍 Scraping Seite 351...
🔍 Scraping Seite 352...
🔍 Scraping Seite 353...
🔍 Scraping Seite 354...
🔍 Scraping Seite 355...
🔍 Scraping Seite 356...
🔍 Scraping Seite 357...
🔍 Scraping Seite 358...
🔍 Scraping Seite 359...
🔍 Scraping Seite 360...
🔍 Scraping Seite 361...
🔍 Scraping Seite 362...
🔍 Scraping Seite 363...
🔍 Scraping Seite 364...
🔍 Scraping Seite 365...
🔍 Scraping Seite 366...
🔍 Scraping Seite 367...
🔍 Scraping Seite 368...
🔍 Scraping Seite 369...
🔍 Scraping Seite 370...
🔍 Scraping Seite 371...
🔍 Scraping Seite 372...
🔍 Scraping Seite 373...
🔍 Scraping Seite 374...
🔍 Scraping Seite

🔍 Scraping Seite 662...
🔍 Scraping Seite 663...
🔍 Scraping Seite 664...
🔍 Scraping Seite 665...
🔍 Scraping Seite 666...
🔍 Scraping Seite 667...
🔍 Scraping Seite 668...
🔍 Scraping Seite 669...
🔍 Scraping Seite 670...
🔍 Scraping Seite 671...
🔍 Scraping Seite 672...
🔍 Scraping Seite 673...
🔍 Scraping Seite 674...
🔍 Scraping Seite 675...
🔍 Scraping Seite 676...
🔍 Scraping Seite 677...
🔍 Scraping Seite 678...
🔍 Scraping Seite 679...
🔍 Scraping Seite 680...
🔍 Scraping Seite 681...
🔍 Scraping Seite 682...
🔍 Scraping Seite 683...
🔍 Scraping Seite 684...
🔍 Scraping Seite 685...
🔍 Scraping Seite 686...
🔍 Scraping Seite 687...
🔍 Scraping Seite 688...
🔍 Scraping Seite 689...
🔍 Scraping Seite 690...
🔍 Scraping Seite 691...
🔍 Scraping Seite 692...
🔍 Scraping Seite 693...
🔍 Scraping Seite 694...
🔍 Scraping Seite 695...
🔍 Scraping Seite 696...
🔍 Scraping Seite 697...
🔍 Scraping Seite 698...
🔍 Scraping Seite 699...
🔍 Scraping Seite 700...
🔍 Scraping Seite 701...
🔍 Scraping Seite 702...
🔍 Scraping Seite

🔍 Scraping Seite 990...
🔍 Scraping Seite 991...
🔍 Scraping Seite 992...
🔍 Scraping Seite 993...
🔍 Scraping Seite 994...
🔍 Scraping Seite 995...
🔍 Scraping Seite 996...
🔍 Scraping Seite 997...
🔍 Scraping Seite 998...
🔍 Scraping Seite 999...
🔍 Scraping Seite 1000...
🔍 Scraping Seite 1001...
🔍 Scraping Seite 1002...
🔍 Scraping Seite 1003...
🔍 Scraping Seite 1004...
🔍 Scraping Seite 1005...
🔍 Scraping Seite 1006...
🔍 Scraping Seite 1007...
🔍 Scraping Seite 1008...
🔍 Scraping Seite 1009...
🔍 Scraping Seite 1010...
🔍 Scraping Seite 1011...
🔍 Scraping Seite 1012...
🔍 Scraping Seite 1013...
🔍 Scraping Seite 1014...
🔍 Scraping Seite 1015...
🔍 Scraping Seite 1016...
🔍 Scraping Seite 1017...
🔍 Scraping Seite 1018...
🔍 Scraping Seite 1019...
🔍 Scraping Seite 1020...
🔍 Scraping Seite 1021...
🔍 Scraping Seite 1022...
🔍 Scraping Seite 1023...
🔍 Scraping Seite 1024...
🔍 Scraping Seite 1025...
🔍 Scraping Seite 1026...
🔍 Scraping Seite 1027...
🔍 Scraping Seite 1028...
🔍 Scraping Seite 1029...
🔍 Scraping

🔍 Scraping Seite 1306...
🔍 Scraping Seite 1307...
🔍 Scraping Seite 1308...
🔍 Scraping Seite 1309...
🔍 Scraping Seite 1310...
🔍 Scraping Seite 1311...
🔍 Scraping Seite 1312...
🔍 Scraping Seite 1313...
🔍 Scraping Seite 1314...
🔍 Scraping Seite 1315...
🔍 Scraping Seite 1316...
🔍 Scraping Seite 1317...
🔍 Scraping Seite 1318...
🔍 Scraping Seite 1319...
🔍 Scraping Seite 1320...
🔍 Scraping Seite 1321...
🔍 Scraping Seite 1322...
🔍 Scraping Seite 1323...
🔍 Scraping Seite 1324...
🔍 Scraping Seite 1325...
🔍 Scraping Seite 1326...
🔍 Scraping Seite 1327...
🔍 Scraping Seite 1328...
🔍 Scraping Seite 1329...
🔍 Scraping Seite 1330...
🔍 Scraping Seite 1331...
🔍 Scraping Seite 1332...
🔍 Scraping Seite 1333...
🔍 Scraping Seite 1334...
🔍 Scraping Seite 1335...
🔍 Scraping Seite 1336...
🔍 Scraping Seite 1337...
🔍 Scraping Seite 1338...
🔍 Scraping Seite 1339...
🔍 Scraping Seite 1340...
🔍 Scraping Seite 1341...
🔍 Scraping Seite 1342...
🔍 Scraping Seite 1343...
🔍 Scraping Seite 1344...
🔍 Scraping Seite 1345...


🔍 Scraping Seite 1622...
🔍 Scraping Seite 1623...
🔍 Scraping Seite 1624...
🔍 Scraping Seite 1625...
🔍 Scraping Seite 1626...
🔍 Scraping Seite 1627...
🔍 Scraping Seite 1628...
🔍 Scraping Seite 1629...
🔍 Scraping Seite 1630...
🔍 Scraping Seite 1631...
🔍 Scraping Seite 1632...
🔍 Scraping Seite 1633...
🔍 Scraping Seite 1634...
🔍 Scraping Seite 1635...
🔍 Scraping Seite 1636...
🔍 Scraping Seite 1637...
🔍 Scraping Seite 1638...
🔍 Scraping Seite 1639...
🔍 Scraping Seite 1640...
🔍 Scraping Seite 1641...
🔍 Scraping Seite 1642...
🔍 Scraping Seite 1643...
🔍 Scraping Seite 1644...
🔍 Scraping Seite 1645...
🔍 Scraping Seite 1646...
🔍 Scraping Seite 1647...
🔍 Scraping Seite 1648...
🔍 Scraping Seite 1649...
🔍 Scraping Seite 1650...
🔍 Scraping Seite 1651...
🔍 Scraping Seite 1652...
🔍 Scraping Seite 1653...
🔍 Scraping Seite 1654...
🔍 Scraping Seite 1655...
🔍 Scraping Seite 1656...
🔍 Scraping Seite 1657...
🔍 Scraping Seite 1658...
🔍 Scraping Seite 1659...
🔍 Scraping Seite 1660...
🔍 Scraping Seite 1661...


🔍 Scraping Seite 1938...
🔍 Scraping Seite 1939...
🔍 Scraping Seite 1940...
🔍 Scraping Seite 1941...
🔍 Scraping Seite 1942...
🔍 Scraping Seite 1943...
🔍 Scraping Seite 1944...
🔍 Scraping Seite 1945...
🔍 Scraping Seite 1946...
🔍 Scraping Seite 1947...
🔍 Scraping Seite 1948...
🔍 Scraping Seite 1949...
🔍 Scraping Seite 1950...
🔍 Scraping Seite 1951...
🔍 Scraping Seite 1952...
🔍 Scraping Seite 1953...
🔍 Scraping Seite 1954...
🔍 Scraping Seite 1955...
🔍 Scraping Seite 1956...
🔍 Scraping Seite 1957...
🔍 Scraping Seite 1958...
🔍 Scraping Seite 1959...
🔍 Scraping Seite 1960...
🔍 Scraping Seite 1961...
🔍 Scraping Seite 1962...
🔍 Scraping Seite 1963...
🔍 Scraping Seite 1964...
🔍 Scraping Seite 1965...
🔍 Scraping Seite 1966...
🔍 Scraping Seite 1967...
🔍 Scraping Seite 1968...
🔍 Scraping Seite 1969...
🔍 Scraping Seite 1970...
🔍 Scraping Seite 1971...
🔍 Scraping Seite 1972...
🔍 Scraping Seite 1973...
🔍 Scraping Seite 1974...
🔍 Scraping Seite 1975...
🔍 Scraping Seite 1976...
🔍 Scraping Seite 1977...


In [ ]:
#10.004 Posts, 200 Seiten, bei Versuch 1
#64.583 Posts, 1252 Seiten dann Timeout (von insgesamt 87.640) #Versuch 2
#82.252 Posts, 2000 Seiten #Versuch 3

In [20]:
# Blick in die Daten
import pandas as pd
df = pd.read_csv("factbase_posts_playwright1.csv")
print(df.tail()) #.T

                                                    Text  \
82247  Donald Trump\n@realDonaldTrump •  Truth Social...   
82248  Donald Trump\n@realDonaldTrump •  Truth Social...   
82249  Donald Trump\n@realDonaldTrump •  Truth Social...   
82250  Donald Trump\n@realDonaldTrump •  Truth Social...   
82251  THE SOURCE FOR NEWS ON\nCAPITOL HILL SINCE 195...   

                                          Screenshot_URL  \
82247  https://media-cdn.factba.se/realdonaldtrump-tr...   
82248  https://media-cdn.factba.se/realdonaldtrump-tr...   
82249  https://media-cdn.factba.se/realdonaldtrump-tr...   
82250  https://media-cdn.factba.se/realdonaldtrump-tr...   
82251                                                NaN   

                             ALT_Text Video_erkannt  
82247  Media from Donald Trump's post            Ja  
82248  Media from Donald Trump's post            Ja  
82249  Media from Donald Trump's post            Ja  
82250  Media from Donald Trump's post            Ja  
82251   

### Versuch 2:
- Ergänzung der Zeitstempel
- Text enthält noch Autor, Plattform, Datum und Uhrzeit,..
- Test, wie viele Posts gefunden werden

In [10]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Für Jupyter-Umgebungen

def is_probable_video(image_url: str, alt_text: str) -> bool:
    """Einfache Videoerkennung."""
    if not image_url:
        return False
    return (
        "video" in image_url.lower()
        or ".mp4" in image_url.lower()
        or "watch" in image_url.lower()
        or "play" in alt_text.lower()
        or "media from" in alt_text.lower()
    )

def extract_timestamp_from_text(text: str) -> str:
    """Extrahiert Datum aus dem Posttext, z. B. 'July 25, 2025'."""
    match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", text)
    return match.group(0) if match else ""

async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 3000

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping Seite {page_num}...")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=30000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()

                    image_el = await block.query_selector("img")
                    image_url = await image_el.get_attribute("src") if image_el else ""
                    alt_text = await image_el.get_attribute("alt") if image_el else ""

                    has_video = is_probable_video(image_url, alt_text)
                    timestamp = extract_timestamp_from_text(full_text)

                    posts_data.append({
                        "Zeitstempel": timestamp,
                        "Text": full_text.strip(),
                        "ALT_Text": alt_text.strip() if alt_text else "",
                        "Video_erkannt": "Ja" if has_video else "Nein"
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright2.csv", index=False, encoding="utf-8-sig")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright2.csv'.")

# Für Notebook:
await scrape_factbase()

# Für .py-Skript (wenn nicht in Jupyter):
# asyncio.run(scrape_factbase())


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...
Scraping Seite 31...
Scraping Seite 32...
Scraping Seite 33...
Scraping Seite 34...
Scraping Seite 35...
Scraping Seite 36...
Scraping Seite 37...
Scraping Seite 38...
Scraping Seite 39...
Scraping Seite 40...
Scraping Seite 41...
Scraping Seite 42...
Scraping Seite 43...
Scraping Seite 44...
Scraping Seite 45...
Scraping Seite 46...
Scraping Seite 47...
Scraping Seite 48...
S

Scraping Seite 379...
Scraping Seite 380...
Scraping Seite 381...
Scraping Seite 382...
Scraping Seite 383...
Scraping Seite 384...
Scraping Seite 385...
Scraping Seite 386...
Scraping Seite 387...
Scraping Seite 388...
Scraping Seite 389...
Scraping Seite 390...
Scraping Seite 391...
Scraping Seite 392...
Scraping Seite 393...
Scraping Seite 394...
Scraping Seite 395...
Scraping Seite 396...
Scraping Seite 397...
Scraping Seite 398...
Scraping Seite 399...
Scraping Seite 400...
Scraping Seite 401...
Scraping Seite 402...
Scraping Seite 403...
Scraping Seite 404...
Scraping Seite 405...
Scraping Seite 406...
Scraping Seite 407...
Scraping Seite 408...
Scraping Seite 409...
Scraping Seite 410...
Scraping Seite 411...
Scraping Seite 412...
Scraping Seite 413...
Scraping Seite 414...
Scraping Seite 415...
Scraping Seite 416...
Scraping Seite 417...
Scraping Seite 418...
Scraping Seite 419...
Scraping Seite 420...
Scraping Seite 421...
Scraping Seite 422...
Scraping Seite 423...
Scraping S

Scraping Seite 752...
Scraping Seite 753...
Scraping Seite 754...
Scraping Seite 755...
Scraping Seite 756...
Scraping Seite 757...
Scraping Seite 758...
Scraping Seite 759...
Scraping Seite 760...
Scraping Seite 761...
Scraping Seite 762...
Scraping Seite 763...
Scraping Seite 764...
Scraping Seite 765...
Scraping Seite 766...
Scraping Seite 767...
Scraping Seite 768...
Scraping Seite 769...
Scraping Seite 770...
Scraping Seite 771...
Scraping Seite 772...
Scraping Seite 773...
Scraping Seite 774...
Scraping Seite 775...
Scraping Seite 776...
Scraping Seite 777...
Scraping Seite 778...
Scraping Seite 779...
Scraping Seite 780...
Scraping Seite 781...
Scraping Seite 782...
Scraping Seite 783...
Scraping Seite 784...
Scraping Seite 785...
Scraping Seite 786...
Scraping Seite 787...
Scraping Seite 788...
Scraping Seite 789...
Scraping Seite 790...
Scraping Seite 791...
Scraping Seite 792...
Scraping Seite 793...
Scraping Seite 794...
Scraping Seite 795...
Scraping Seite 796...
Scraping S

Scraping Seite 1119...
Scraping Seite 1120...
Scraping Seite 1121...
Scraping Seite 1122...
Scraping Seite 1123...
Scraping Seite 1124...
Scraping Seite 1125...
Scraping Seite 1126...
Scraping Seite 1127...
Scraping Seite 1128...
Scraping Seite 1129...
Scraping Seite 1130...
Scraping Seite 1131...
Scraping Seite 1132...
Scraping Seite 1133...
Scraping Seite 1134...
Scraping Seite 1135...
Scraping Seite 1136...
Scraping Seite 1137...
Scraping Seite 1138...
Scraping Seite 1139...
Scraping Seite 1140...
Scraping Seite 1141...
Scraping Seite 1142...
Scraping Seite 1143...
Scraping Seite 1144...
Scraping Seite 1145...
Scraping Seite 1146...
Scraping Seite 1147...
Scraping Seite 1148...
Scraping Seite 1149...
Scraping Seite 1150...
Scraping Seite 1151...
Scraping Seite 1152...
Scraping Seite 1153...
Scraping Seite 1154...
Scraping Seite 1155...
Scraping Seite 1156...
Scraping Seite 1157...
Scraping Seite 1158...
Scraping Seite 1159...
Scraping Seite 1160...
Scraping Seite 1161...
Scraping Se

Scraping Seite 1476...
Scraping Seite 1477...
Scraping Seite 1478...
Scraping Seite 1479...
Scraping Seite 1480...
Scraping Seite 1481...
Scraping Seite 1482...
Scraping Seite 1483...
Scraping Seite 1484...
Scraping Seite 1485...
Scraping Seite 1486...
Scraping Seite 1487...
Scraping Seite 1488...
Scraping Seite 1489...
Scraping Seite 1490...
Scraping Seite 1491...
Scraping Seite 1492...
Scraping Seite 1493...
Scraping Seite 1494...
Scraping Seite 1495...
Scraping Seite 1496...
Scraping Seite 1497...
Scraping Seite 1498...
Scraping Seite 1499...
Scraping Seite 1500...
Scraping Seite 1501...
Scraping Seite 1502...
Scraping Seite 1503...
Scraping Seite 1504...
Scraping Seite 1505...
Scraping Seite 1506...
Scraping Seite 1507...
Scraping Seite 1508...
Scraping Seite 1509...
Scraping Seite 1510...
Scraping Seite 1511...
Scraping Seite 1512...
Scraping Seite 1513...
Scraping Seite 1514...
Scraping Seite 1515...
Scraping Seite 1516...
Scraping Seite 1517...
Scraping Seite 1518...
Scraping Se

Scraping Seite 1833...
Scraping Seite 1834...
Scraping Seite 1835...
Scraping Seite 1836...
Scraping Seite 1837...
Scraping Seite 1838...
Scraping Seite 1839...
Scraping Seite 1840...
Scraping Seite 1841...
Scraping Seite 1842...
Scraping Seite 1843...
Scraping Seite 1844...
Scraping Seite 1845...
Scraping Seite 1846...
Scraping Seite 1847...
Scraping Seite 1848...
Scraping Seite 1849...
Scraping Seite 1850...
Scraping Seite 1851...
Scraping Seite 1852...
Scraping Seite 1853...
Scraping Seite 1854...
Scraping Seite 1855...
Scraping Seite 1856...
Scraping Seite 1857...
Scraping Seite 1858...
Scraping Seite 1859...
Scraping Seite 1860...
Scraping Seite 1861...
Scraping Seite 1862...
Scraping Seite 1863...
Scraping Seite 1864...
Scraping Seite 1865...
Scraping Seite 1866...
Scraping Seite 1867...
Scraping Seite 1868...
Scraping Seite 1869...
Scraping Seite 1870...
Scraping Seite 1871...
Scraping Seite 1872...
Scraping Seite 1873...
Scraping Seite 1874...
Scraping Seite 1875...
Scraping Se

Scraping Seite 2190...
Scraping Seite 2191...
Scraping Seite 2192...
Scraping Seite 2193...
Scraping Seite 2194...
Scraping Seite 2195...
Scraping Seite 2196...
Scraping Seite 2197...
Scraping Seite 2198...
Scraping Seite 2199...
Scraping Seite 2200...
Scraping Seite 2201...
Scraping Seite 2202...
Scraping Seite 2203...
Scraping Seite 2204...
Scraping Seite 2205...
Scraping Seite 2206...
Scraping Seite 2207...
Scraping Seite 2208...
Scraping Seite 2209...
Scraping Seite 2210...
Scraping Seite 2211...
Scraping Seite 2212...
Scraping Seite 2213...
Scraping Seite 2214...
Scraping Seite 2215...
Scraping Seite 2216...
Scraping Seite 2217...
Scraping Seite 2218...
Scraping Seite 2219...
Scraping Seite 2220...
Scraping Seite 2221...
Scraping Seite 2222...
Scraping Seite 2223...
Scraping Seite 2224...
Scraping Seite 2225...
Scraping Seite 2226...
Scraping Seite 2227...
Scraping Seite 2228...
Scraping Seite 2229...
Scraping Seite 2230...
Scraping Seite 2231...
Scraping Seite 2232...
Scraping Se

Scraping Seite 2547...
Scraping Seite 2548...
Scraping Seite 2549...
Scraping Seite 2550...
Scraping Seite 2551...
Scraping Seite 2552...
Scraping Seite 2553...
Scraping Seite 2554...
Scraping Seite 2555...
Scraping Seite 2556...
Scraping Seite 2557...
Scraping Seite 2558...
Scraping Seite 2559...
Scraping Seite 2560...
Scraping Seite 2561...
Scraping Seite 2562...
Scraping Seite 2563...
Scraping Seite 2564...
Scraping Seite 2565...
Scraping Seite 2566...
Scraping Seite 2567...
Scraping Seite 2568...
Scraping Seite 2569...
Scraping Seite 2570...
Scraping Seite 2571...
Scraping Seite 2572...
Scraping Seite 2573...
Scraping Seite 2574...
Scraping Seite 2575...
Scraping Seite 2576...
Scraping Seite 2577...
Scraping Seite 2578...
Scraping Seite 2579...
Scraping Seite 2580...
Scraping Seite 2581...
Scraping Seite 2582...
Scraping Seite 2583...
Scraping Seite 2584...
Scraping Seite 2585...
Scraping Seite 2586...
Scraping Seite 2587...
Scraping Seite 2588...
Scraping Seite 2589...
Scraping Se

In [ ]:
#127.966 Posts, Fehler beim Laden von Seite 2615
# nur die letzten 4 Posts sind leer??
# Wie viele Posts gibt es wirklich?
#(Auf der Website waren am 1.8. 87.640 angegeben.)

In [ ]:
import pandas as pd
fp = pd.read_csv("factbase_posts_playwright2.csv")
#print(fpdp.tail(10))
#print(fpp[600-604])
print(fp.shape)

In [ ]:
# Blick in die Daten:

In [16]:
import pandas as pd
fpdp = pd.read_csv("factbase_posts_playwright2.csv")
print(fpdp.tail(20))

           Zeitstempel                                               Text  \
127946  August 2, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127947  August 2, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127948  August 2, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127949  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127950  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127951  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127952  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127953  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127954  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127955  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127956  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   
127957  August 1, 2025  Donald Trump\n@realDonaldTrump •  Truth Social...   

In [ ]:
# Der Zeitstempel funktioniert ganz okay, der Rest noch nicht so.
# Datum, Uhrzeit und co. sind im Textobjekt gespeichert und nicht explizit gelabelt.

Auf Truth Social sind alle Metadaten einzeln gelabelt
-> kurzzeitige Überlegung, es doch direkt mit Truthsocial zu versuchen, da ich zunächst Schwierigkeiten hatte, die richtigen Objekte der HTML-Seite zu finden
(und von Twitter gibt es die Daten bereits im Trump-Twitter-Archiv).

In [ ]:
auf truthsocial: text -> div.relative
    Sprechblasen: p class: text-xs text-inherit font-semibold tracking-normal font-sans normal-case [&_span.invisible]:inline-block [&_span.invisible]:w-0
    Retweets: div.flex rtl:space-x-reverse items-center justify-center group rounded-full rtl:space-x-reverse text-gray-600
    Likes: div.flex rtl:space-x-reverse items-center justify-center group rounded-full rtl:space-x-reverse text-gray-600

In [ ]:
aria-label: "Like"
aria-label: "ReTruth"
aria-label: "Reply"
dat: a href

Allerdings sind auf Truth Social immer nur die neuesten Posts sichtbar.
Ältere Posts werden gelöscht oder sind nur mit angelegtem Profil sichtbar. (und weitere Probleme)

#### Ab jetzt limitiere ich den Code auf 3 Seiten, damit der Code nicht ewig lädt (kann nätürlich beliebig geändert werden).

#### Textbeispiel, um den Code richtig anzupassen:
- Autor, Plattform, Datum und Uhrzeit sind ebenfalls im Text enthalten
- reiner Text im "text" wäre cool

In [ ]:
sample_text = """Donald Trump
@realDonaldTrump •  Truth Social • August 1, 2025 @ 6:05 PM ET
View on Truth Social

“Too Late” Powell should resign, just like Adriana Kugler, a Biden Appointee, resigned. She knew he was doing the wrong thing on Interest Rates. He should resign, also!"""

In [11]:
#neue Funktion schreiben, die mit Datum usw. richtig umgehen kann:
def extract_metadata_test(text: str):
    """
    Extrahiert Autor (mit Handle), Plattform, Datum, Uhrzeit, Tag, Monat, Jahr und den Text des Posts.
    """
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    # Struktur:
    # Zeile 0: Donald Trump
    # Zeile 1: @realDonaldTrump •  Truth Social • August 1, 2025 @ 6:05 PM ET
    # Zeile 2: View on Truth Social
    # Ab Zeile 3: eigentlicher Inhalt

    if len(lines) >= 3:
        author_name = lines[0].strip()
        
        # z.B.: @realDonaldTrump • Truth Social • August 1, 2025 @ 6:05 PM ET
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Restlicher Inhalt
        content_lines = lines[3:]  # Post-Inhalt beginnt ab Zeile 3

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except Exception as e:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

In [12]:
meta = extract_metadata_test(sample_text)
print(meta)


{'author': 'Donald Trump @realDonaldTrump', 'platform': 'Truth Social', 'date': '2025-08-01', 'time': '18:05', 'year': 2025, 'month': 'August', 'day': 1, 'text': '“Too Late” Powell should resign, just like Adriana Kugler, a Biden Appointee, resigned. She knew he was doing the wrong thing on Interest Rates. He should resign, also!'}


In [15]:
import pandas as pd
md = pd.DataFrame([meta])
print(md.T)

                                                          0
author                        Donald Trump @realDonaldTrump
platform                                       Truth Social
date                                             2025-08-01
time                                                  18:05
year                                                   2025
month                                                August
day                                                       1
text      “Too Late” Powell should resign, just like Adr...


In [19]:
print(md.text[0])

“Too Late” Powell should resign, just like Adriana Kugler, a Biden Appointee, resigned. She knew he was doing the wrong thing on Interest Rates. He should resign, also!


..und jetzt im ganzen Code...

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Für Jupyter-Umgebungen

def is_probable_video(image_url: str, alt_text: str) -> bool:
    """Einfache Videoerkennung."""
    if not image_url:
        return False
    return (
        "video" in image_url.lower()
        or ".mp4" in image_url.lower()
        or "watch" in image_url.lower()
        or "play" in alt_text.lower()
        or "media from" in alt_text.lower()
    )

#def extract_timestamp_from_text(text: str) -> str:
#    """Extrahiert Datum aus dem Posttext, z. B. 'July 25, 2025'."""
#    match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", text)
#    return match.group(0) if match else ""


def extract_metadata_text(text: str):
    """
    Extrahiert Autor (mit Handle), Plattform, Datum, Uhrzeit, Tag, Monat, Jahr und reinen Post-Text.
    """
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    # Struktur:
    # Zeile 0: Donald Trump
    # Zeile 1: @realDonaldTrump •  Truth Social • August 1, 2025 @ 6:05 PM ET
    # Zeile 2: View on Truth Social
    # Ab Zeile 3: eigentlicher Inhalt

    if len(lines) >= 3:
        author_name = lines[0].strip()
        
        # Match z.B.: @realDonaldTrump • Truth Social • August 1, 2025 @ 6:05 PM ET
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Restlicher Inhalt
        content_lines = lines[3:]  # Post-Inhalt beginnt ab Zeile 3

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except Exception as e:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }


async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 3 #3000
    #erstmal mit kleineren Zahlen testen, damit das Programm nicht ewig lädt.

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping Seite {page_num}...")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=30000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()

                    image_el = await block.query_selector("img")
                    image_url = await image_el.get_attribute("src") if image_el else ""
                    alt_text = await image_el.get_attribute("alt") if image_el else ""

                    has_video = is_probable_video(image_url, alt_text)
                    #timestamp = extract_timestamp_from_text(full_text)
                    
                    data = extract_metadata_text(full_text)


                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        #"Zeitstempel": timestamp,
                        
                        "transcript": alt_text.strip() if alt_text else "",
                        "video": "Ja" if has_video else "Nein"
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright3.csv", index=False, encoding="utf-8") #-sig?
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright3.csv'.")

# Für Notebook:
await scrape_factbase()

Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...


In [26]:
import pandas as pd
fpdp = pd.read_csv("factbase_posts_playwright3.csv")
#print(fpdp.tail(10))
print(fpdp.text[8])

https://www.foxnews.com/politics/exclusive-trump-admin-nixes-giant-wind-farm-approved-last-minute-biden-team


In [24]:
print(fpdp.head(10))

                          author      platform        date   time  day  \
0  Donald Trump @realDonaldTrump  Truth Social  2025-08-07  07:39  7.0   
1  Donald Trump @realDonaldTrump  Truth Social  2025-08-07  07:35  7.0   
2  Donald Trump @realDonaldTrump  Truth Social  2025-08-07  07:31  7.0   
3  Donald Trump @realDonaldTrump  Truth Social  2025-08-07  07:22  7.0   
4  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  23:58  6.0   
5  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  23:44  6.0   
6  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  18:04  6.0   
7  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  18:01  6.0   
8  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  18:00  6.0   
9  Donald Trump @realDonaldTrump  Truth Social  2025-08-06  17:59  6.0   

    month    year                                               text  \
0  August  2025.0  View on Truth Social\nThe CEO of INTEL is high...   
1  August  2025.0  View on Truth Social\n

### Funktionierender Versuch
- richtiges Datum
- richtiger Inhalt des Textes
- richtiges Trennen von Datum, Plattform, Autor usw.
- richtiges Speichern der Daten
- aber: zu viele Posts? (202.000) der Code würde ewig laufen

In [1]:
## Dauer, bis das Programm fertig ist: ca. 5 h
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Für Jupyter-Umgebungen

#Folgende Funktion ist nicht benutzbar
#def is_probable_video(image_url: str, alt_text: str) -> bool:
#    """Einfache Videoerkennung."""
#    if not image_url:
#        return False
#    return (
#        "video" in image_url.lower()
#        or ".mp4" in image_url.lower()
#        or "watch" in image_url.lower()
#        or "play" in alt_text.lower()
#        or "media from" in alt_text.lower()
#    )

#def extract_timestamp_from_text(text: str) -> str:
#    """Extrahiert Datum aus dem Posttext, z. B. 'July 25, 2025'."""
#    match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", text)
#    return match.group(0) if match else ""


def extract_metadata_text(text: str):
    """
    Extrahiert Autor (mit Handle), Plattform, Datum, Uhrzeit, Tag, Monat, Jahr und reinen Post-Text.
    """
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    # Struktur:
    # Zeile 0: Donald Trump
    # Zeile 1: @realDonaldTrump •  Truth Social • August 1, 2025 @ 6:05 PM ET
    # Zeile 2: View on Truth Social
    # Ab Zeile 3: eigentlicher Inhalt

    if len(lines) >= 3:
        author_name = lines[0].strip()
        
        # Match z.B.: @realDonaldTrump • Truth Social • August 1, 2025 @ 6:05 PM ET
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Restlicher Inhalt
        content_lines = lines[3:]  # Post-Inhalt beginnt ab Zeile 3

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except Exception as e:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }


async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 4000
    #erstmal mit kleineren Zahlen testen, damit das Programm nicht ewig lädt.

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()

                    #image_el = await block.query_selector("img")
                    #image_url = await image_el.get_attribute("src") if image_el else ""
                    #alt_text = await image_el.get_attribute("alt") if image_el else ""

                    #has_video = is_probable_video(image_url, alt_text)
                    #timestamp = extract_timestamp_from_text(full_text)
                    
                    data = extract_metadata_text(full_text)


                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        #"Zeitstempel": timestamp,
                        
                        #"transcript": alt_text.strip() if alt_text else "",
                        #"video": "Ja" if has_video else "Nein"
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright5.csv", index=False, encoding="utf-8")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright5.csv'.")

# Für Notebook:
await scrape_factbase()

Scraping von Seite 1
Scraping von Seite 2
Scraping von Seite 3
Scraping von Seite 4
Scraping von Seite 5
Scraping von Seite 6
Scraping von Seite 7
Scraping von Seite 8
Scraping von Seite 9
Scraping von Seite 10
Scraping von Seite 11
Scraping von Seite 12
Scraping von Seite 13
Scraping von Seite 14
Scraping von Seite 15
Scraping von Seite 16
Scraping von Seite 17
Scraping von Seite 18
Scraping von Seite 19
Scraping von Seite 20
Scraping von Seite 21
Scraping von Seite 22
Scraping von Seite 23
Scraping von Seite 24
Scraping von Seite 25
Scraping von Seite 26
Scraping von Seite 27
Scraping von Seite 28
Scraping von Seite 29
Scraping von Seite 30
Scraping von Seite 31
Scraping von Seite 32
Scraping von Seite 33
Scraping von Seite 34
Scraping von Seite 35
Scraping von Seite 36
Scraping von Seite 37
Scraping von Seite 38
Scraping von Seite 39
Scraping von Seite 40
Scraping von Seite 41
Scraping von Seite 42
Scraping von Seite 43
Scraping von Seite 44
Scraping von Seite 45
Scraping von Seite 

Scraping von Seite 362
Scraping von Seite 363
Scraping von Seite 364
Scraping von Seite 365
Scraping von Seite 366
Scraping von Seite 367
Scraping von Seite 368
Scraping von Seite 369
Scraping von Seite 370
Scraping von Seite 371
Scraping von Seite 372
Scraping von Seite 373
Scraping von Seite 374
Scraping von Seite 375
Scraping von Seite 376
Scraping von Seite 377
Scraping von Seite 378
Scraping von Seite 379
Scraping von Seite 380
Scraping von Seite 381
Scraping von Seite 382
Scraping von Seite 383
Scraping von Seite 384
Scraping von Seite 385
Scraping von Seite 386
Scraping von Seite 387
Scraping von Seite 388
Scraping von Seite 389
Scraping von Seite 390
Scraping von Seite 391
Scraping von Seite 392
Scraping von Seite 393
Scraping von Seite 394
Scraping von Seite 395
Scraping von Seite 396
Scraping von Seite 397
Scraping von Seite 398
Scraping von Seite 399
Scraping von Seite 400
Scraping von Seite 401
Scraping von Seite 402
Scraping von Seite 403
Scraping von Seite 404
Scraping vo

Scraping von Seite 719
Scraping von Seite 720
Scraping von Seite 721
Scraping von Seite 722
Scraping von Seite 723
Scraping von Seite 724
Scraping von Seite 725
Scraping von Seite 726
Scraping von Seite 727
Scraping von Seite 728
Scraping von Seite 729
Scraping von Seite 730
Scraping von Seite 731
Scraping von Seite 732
Scraping von Seite 733
Scraping von Seite 734
Scraping von Seite 735
Scraping von Seite 736
Scraping von Seite 737
Scraping von Seite 738
Scraping von Seite 739
Scraping von Seite 740
Scraping von Seite 741
Scraping von Seite 742
Scraping von Seite 743
Scraping von Seite 744
Scraping von Seite 745
Scraping von Seite 746
Scraping von Seite 747
Scraping von Seite 748
Scraping von Seite 749
Scraping von Seite 750
Scraping von Seite 751
Scraping von Seite 752
Scraping von Seite 753
Scraping von Seite 754
Scraping von Seite 755
Scraping von Seite 756
Scraping von Seite 757
Scraping von Seite 758
Scraping von Seite 759
Scraping von Seite 760
Scraping von Seite 761
Scraping vo

Scraping von Seite 1073
Scraping von Seite 1074
Scraping von Seite 1075
Scraping von Seite 1076
Scraping von Seite 1077
Scraping von Seite 1078
Scraping von Seite 1079
Scraping von Seite 1080
Scraping von Seite 1081
Scraping von Seite 1082
Scraping von Seite 1083
Scraping von Seite 1084
Scraping von Seite 1085
Scraping von Seite 1086
Scraping von Seite 1087
Scraping von Seite 1088
Scraping von Seite 1089
Scraping von Seite 1090
Scraping von Seite 1091
Scraping von Seite 1092
Scraping von Seite 1093
Scraping von Seite 1094
Scraping von Seite 1095
Scraping von Seite 1096
Scraping von Seite 1097
Scraping von Seite 1098
Scraping von Seite 1099
Scraping von Seite 1100
Scraping von Seite 1101
Scraping von Seite 1102
Scraping von Seite 1103
Scraping von Seite 1104
Scraping von Seite 1105
Scraping von Seite 1106
Scraping von Seite 1107
Scraping von Seite 1108
Scraping von Seite 1109
Scraping von Seite 1110
Scraping von Seite 1111
Scraping von Seite 1112
Scraping von Seite 1113
Scraping von Sei

Scraping von Seite 1415
Scraping von Seite 1416
Scraping von Seite 1417
Scraping von Seite 1418
Scraping von Seite 1419
Scraping von Seite 1420
Scraping von Seite 1421
Scraping von Seite 1422
Scraping von Seite 1423
Scraping von Seite 1424
Scraping von Seite 1425
Scraping von Seite 1426
Scraping von Seite 1427
Scraping von Seite 1428
Scraping von Seite 1429
Scraping von Seite 1430
Scraping von Seite 1431
Scraping von Seite 1432
Scraping von Seite 1433
Scraping von Seite 1434
Scraping von Seite 1435
Scraping von Seite 1436
Scraping von Seite 1437
Scraping von Seite 1438
Scraping von Seite 1439
Scraping von Seite 1440
Scraping von Seite 1441
Scraping von Seite 1442
Scraping von Seite 1443
Scraping von Seite 1444
Scraping von Seite 1445
Scraping von Seite 1446
Scraping von Seite 1447
Scraping von Seite 1448
Scraping von Seite 1449
Scraping von Seite 1450
Scraping von Seite 1451
Scraping von Seite 1452
Scraping von Seite 1453
Scraping von Seite 1454
Scraping von Seite 1455
Scraping von Sei

...

Scraping von Seite 2441
Scraping von Seite 2442
Scraping von Seite 2443
Scraping von Seite 2444
Scraping von Seite 2445
Scraping von Seite 2446
Scraping von Seite 2447
Scraping von Seite 2448
Scraping von Seite 2449
Scraping von Seite 2450
Scraping von Seite 2451
Scraping von Seite 2452
Scraping von Seite 2453
Scraping von Seite 2454
Scraping von Seite 2455
Scraping von Seite 2456
Scraping von Seite 2457
Scraping von Seite 2458
Scraping von Seite 2459
Scraping von Seite 2460
Scraping von Seite 2461
Scraping von Seite 2462
Scraping von Seite 2463
Scraping von Seite 2464
Scraping von Seite 2465
Scraping von Seite 2466
Scraping von Seite 2467
Scraping von Seite 2468
Scraping von Seite 2469
Scraping von Seite 2470
Scraping von Seite 2471
Scraping von Seite 2472
Scraping von Seite 2473
Scraping von Seite 2474
Scraping von Seite 2475
Scraping von Seite 2476
Scraping von Seite 2477
Scraping von Seite 2478
Scraping von Seite 2479
Scraping von Seite 2480
Scraping von Seite 2481
Scraping von Sei

Scraping von Seite 2783
Scraping von Seite 2784
Scraping von Seite 2785
Scraping von Seite 2786
Scraping von Seite 2787
Scraping von Seite 2788
Scraping von Seite 2789
Scraping von Seite 2790
Scraping von Seite 2791
Scraping von Seite 2792
Scraping von Seite 2793
Scraping von Seite 2794
Scraping von Seite 2795
Scraping von Seite 2796
Scraping von Seite 2797
Scraping von Seite 2798
Scraping von Seite 2799
Scraping von Seite 2800
Scraping von Seite 2801
Scraping von Seite 2802
Scraping von Seite 2803
Scraping von Seite 2804
Scraping von Seite 2805
Scraping von Seite 2806
Scraping von Seite 2807
Scraping von Seite 2808
Scraping von Seite 2809
Scraping von Seite 2810
Scraping von Seite 2811
Scraping von Seite 2812
Scraping von Seite 2813
Scraping von Seite 2814
Scraping von Seite 2815
Scraping von Seite 2816
Scraping von Seite 2817
Scraping von Seite 2818
Scraping von Seite 2819
Scraping von Seite 2820
Scraping von Seite 2821
Scraping von Seite 2822
Scraping von Seite 2823
Scraping von Sei

Scraping von Seite 3125
Scraping von Seite 3126
Scraping von Seite 3127
Scraping von Seite 3128
Scraping von Seite 3129
Scraping von Seite 3130
Scraping von Seite 3131
Scraping von Seite 3132
Scraping von Seite 3133
Scraping von Seite 3134
Scraping von Seite 3135
Scraping von Seite 3136
Scraping von Seite 3137
Scraping von Seite 3138
Scraping von Seite 3139
Scraping von Seite 3140
Scraping von Seite 3141
Scraping von Seite 3142
Scraping von Seite 3143
Scraping von Seite 3144
Scraping von Seite 3145
Scraping von Seite 3146
Scraping von Seite 3147
Scraping von Seite 3148
Scraping von Seite 3149
Scraping von Seite 3150
Scraping von Seite 3151
Scraping von Seite 3152
Scraping von Seite 3153
Scraping von Seite 3154
Scraping von Seite 3155
Scraping von Seite 3156
Scraping von Seite 3157
Scraping von Seite 3158
Scraping von Seite 3159
Scraping von Seite 3160
Scraping von Seite 3161
Scraping von Seite 3162
Scraping von Seite 3163
Scraping von Seite 3164
Scraping von Seite 3165
Scraping von Sei

Scraping von Seite 3467
Scraping von Seite 3468
Scraping von Seite 3469
Scraping von Seite 3470
Scraping von Seite 3471
Scraping von Seite 3472
Scraping von Seite 3473
Scraping von Seite 3474
Scraping von Seite 3475
Scraping von Seite 3476
Scraping von Seite 3477
Scraping von Seite 3478
Scraping von Seite 3479
Scraping von Seite 3480
Scraping von Seite 3481
Scraping von Seite 3482
Scraping von Seite 3483
Scraping von Seite 3484
Scraping von Seite 3485
Scraping von Seite 3486
Scraping von Seite 3487
Scraping von Seite 3488
Scraping von Seite 3489
Scraping von Seite 3490
Scraping von Seite 3491
Scraping von Seite 3492
Scraping von Seite 3493
Scraping von Seite 3494
Scraping von Seite 3495
Scraping von Seite 3496
Scraping von Seite 3497
Scraping von Seite 3498
Scraping von Seite 3499
Scraping von Seite 3500
Scraping von Seite 3501
Scraping von Seite 3502
Scraping von Seite 3503
Scraping von Seite 3504
Scraping von Seite 3505
Scraping von Seite 3506
Scraping von Seite 3507
Scraping von Sei

Scraping von Seite 3809
Scraping von Seite 3810
Scraping von Seite 3811
Scraping von Seite 3812
Scraping von Seite 3813
Scraping von Seite 3814
Scraping von Seite 3815
Scraping von Seite 3816
Scraping von Seite 3817
Scraping von Seite 3818
Scraping von Seite 3819
Scraping von Seite 3820
Scraping von Seite 3821
Scraping von Seite 3822
Scraping von Seite 3823
Scraping von Seite 3824
Scraping von Seite 3825
Scraping von Seite 3826
Scraping von Seite 3827
Scraping von Seite 3828
Scraping von Seite 3829
Scraping von Seite 3830
Scraping von Seite 3831
Scraping von Seite 3832
Scraping von Seite 3833
Scraping von Seite 3834
Scraping von Seite 3835
Scraping von Seite 3836
Scraping von Seite 3837
Scraping von Seite 3838
Scraping von Seite 3839
Scraping von Seite 3840
Scraping von Seite 3841
Scraping von Seite 3842
Scraping von Seite 3843
Scraping von Seite 3844
Scraping von Seite 3845
Scraping von Seite 3846
Scraping von Seite 3847
Scraping von Seite 3848
Scraping von Seite 3849
Scraping von Sei

In [ ]:
import pandas as pd
fpp = pd.read_csv("factbase_posts_playwright5.csv")
#print(fpdp.tail(10))
print(fpp[600:604].T)

In [ ]:
print(fpp.tail())

In [ ]:
print(fpp.shape)

In [ ]:
trump_posts = fpp[fpp['author'].str.contains("Donald Trump", na=False)]

print(f"Anzahl Trump-Posts: {len(trump_posts)}")

In [ ]:
print(fpp.author.tail(25))

# Fail: 1. Versuch mit Selectolax

Selectolax kann nur mit statischen Websites umgehen, ist aber deutlich schneller als bspw. Beautifulsoup.

In [ ]:
import httpx
from selectolax.parser import HTMLParser
import time
from urllib.parse import urljoin
from dataclasses import dataclass, asdict, fields
import json
import csv

In [ ]:
@dataclass
class item:
    name: str | None
    item_num: str | None
    price: str | None
    rating: float | None

In [ ]:
def get_html(url, **kwargs):
    #User-Agent vor dem Ausführen des Scripts ändern: 
    header={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15"}
    if kwargs.get("page"):
        resp = httpx.get(url + str(kwargs.get("page")), headers=header, follow_redirects=True)
    else:
        resp = httpx.get(url, headers=header, follow_redirect=True)
    try:
        resp.raise_for_status()
    except HTTPStatusError as exc:
        print(f"Error response {exc.resp.status_code} while requesting {exc.request_url!r}. Page Limit Reached.")
        return False
    html = HTMLParser(resp.text)
    return html

In [ ]:
#header={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15"}

In [ ]:
#resp = httpx.get(baseurl + str(page), headers=header, follow_redirects=True)

In [ ]:
print(resp)

In [ ]:
print(resp.status_code) 
#hiermit testen, ob die Seite noch antwortet 
#(200 passt, 301 redirect, 404 fehler, 501 geblockt)

In [ ]:
print(resp.text) #Quellcode der URL

In [ ]:
html = HTMLParser(resp.text)
return html

In [ ]:
def parse_single_tweets(html): 
    #muss angepasst werden (es sind auch die falschen HTML-Elemente ausgewählt)
    new_item = Item{
        name = extract_text(html, "h1#product-page-title"),
        screenshot = extract_text(html, "div#w-full md:w-1/3"),
        price = extract_text(html, "span#buy-box-product-price"),
        rating = extract_text(html, "span#cor-rating-number-13-5-3")
    }
    return asdict(new_item)

In [ ]:
def extract_text(html, sel): #mini-pipeline: cleaning text and giving it back
    try:
        text = html.css_first(sel).text()
        return clean_data(text)
    except: AttributeError:
            return None

In [ ]:
print(html.css_first("title"))

In [ ]:
print(html.css_first("title").text())

In [ ]:
def parse_tweets(html):#div.results-container
    tweets = html.css("div.block mb-8 rounded-xl border border-[#2F3C4B]/20") #block mb-8 rounded-xl border border-[#2F3C4B]/20
    for tweet in tweets:
        yield urljoin("https://rollcall.com/factbase-twitter/", tweet.css_first("a").attributes["href"])
        # href muss noch angepasst werden

In [ ]:
tweets=html.css("div.results-container") #p-4?
print(tweets) #bei Klassen mit Leerzeichen: den ersten Teil wählen oder: 
# die Klasse auswählen und daraus das Attribut verwenden:
# tweet.css_first("div[data-ui=sale-price]").text()

entweder als json:

In [ ]:
def export_to_json(tweets):
    with open("trumptweets.json", "w", encoding="utf-8") as f:
        json.dump(tweets, f, ensure_ascii=False, indent=4) #dumps -> in einen String
    print("Saved to json")

oder als csv:

In [ ]:
def export_to_csv(tweets):
    field_names = [field.name for field in fields(Item)]
    with open("trumptweets.csv", "w", encoding="utf-8") as f:
        writer = csv.DictWriter(f, field_names)
        writer.writeheader()
        writer.writerows(tweets)
    print("Saved to csv")

In [ ]:
def append_to_csv(tweet):
    field_names = [field.name for field in fields(Item)]
    with open("appendcsv.csv", "a") as f:
        writer = csv.DictWriter(f, field_names)
        writer.writerow(tweet)
        #dann bei main(): keine tweets = [] nötig und 
        #kein tweets.append(parse_single_tweets(html))
        # aber: append_to_csv(parse_single_tweets(html))

In [ ]:
def clean_data(value):
    chars_to_remove = ["€", "Item", "#"]
    for char in chars_to_remove:
        if char in value:
            value = value.replace(char, "")
    return value.strip()

In [ ]:
#!!!Code funktioniert nicht
def main():
    tweets = []
    baseurl = "https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page="
    for x in range(1, 1000):
        print(f"Gathering page: {x}")
        html = get_html(baseurl, page=x)
        if html is False:
            break
        tweet_urls = parse_tweets(html)
        for url in tweet_urls:
            print(url)
            html = get_html(url)
            tweets.append(parse_single_tweets(html))
            time.sleep(1)
    export_to_json(tweets)
    export_to_csv(tweets)

In [ ]:
if __name__ == "__main__":
    main()

# Finaler Code:

In [ ]:
### Lösung des Problems mit der Anzahl der Posts?

In [4]:
## Dauer, bis das Programm fertig ist: ca. 5 h
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Für Jupyter-Umgebungen

def extract_metadata_text(text: str):
    """
    Extrahiert Autor (mit Handle), Plattform, Datum, Uhrzeit, Tag, Monat, Jahr und reinen Post-Text.
    """
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    # Struktur:
    # Zeile 0: Donald Trump
    # Zeile 1: @realDonaldTrump •  Truth Social • August 1, 2025 @ 6:05 PM ET
    # Zeile 2: View on Truth Social
    # Ab Zeile 3: eigentlicher Inhalt

    if len(lines) >= 3:
        author_name = lines[0].strip()
        
        # Match z.B.: @realDonaldTrump • Truth Social • August 1, 2025 @ 6:05 PM ET
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Restlicher Inhalt
        content_lines = lines[3:]  # Post-Inhalt beginnt ab Zeile 3

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except Exception as e:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }


async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 4000


    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)


                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright5.csv", index=False, encoding="utf-8")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright5.csv'.")

# Für Notebook:
await scrape_factbase()

In [2]:
import pandas as pd
ut = pd.read_csv("factbase_posts_playwright5.csv")
print(ut.tail(10).T)
#print(ut[600:604].T)

                                                          198  \
author                          Donald Trump @realDonaldTrump   
platform                                         Truth Social   
date                                               2025-08-10   
time                                                    11:03   
day                                                      10.0   
month                                                  August   
year                                                   2025.0   
text        We’re having a News Conference tomorrow in the...   
image_path                                                NaN   
id                                                        199   

                                      199  \
author      Donald Trump @realDonaldTrump   
platform                     Truth Social   
date                           2025-08-10   
time                                11:11   
day                                  10.0   
month         

In [3]:
ut.shape

(208, 10)

In [4]:
ut_unique = ut.drop_duplicates(subset=['time','text'])

ut_unique.to_csv("factbase_posts_playwright5", index=False)

print(f"Vorher: {len(ut)} Zeilen, danach: {len(ut_unique)} Zeilen ohne Duplikate.")


Vorher: 208 Zeilen, danach: 51 Zeilen ohne Duplikate.


In [ ]:
## Umgang mit Duplikaten?

# Umgang mit Screenshots

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
import os
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import ssl
from PIL import Image
import pytesseract

nest_asyncio.apply()

# Falls Tesseract nicht im PATH ist, hier Pfad anpassen (Windows-Beispiel):
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# SSL-Kontext für aiohttp (ohne Zertifikatsprüfung)
sslcontext = ssl.create_default_context()
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

os.makedirs("images", exist_ok=True)

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()
        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Fehler bei OCR von {image_path}: {e}")
        return ""

def parse_metrics(text):
    retweets = re.search(r"Retweet[s]?:?\s*(\d+)", text, re.IGNORECASE)
    likes = re.search(r"Like[s]?:?\s*(\d+)", text, re.IGNORECASE)
    replies = re.search(r"(Reply|Antwort)[s]?:?\s*(\d+)", text, re.IGNORECASE)

    def safe_int(match):
        return int(match.group(1)) if match else None

    return {
        "retweets": safe_int(retweets),
        "likes": safe_int(likes),
        "replies": safe_int(replies)
    }

async def scrape_factbase():
    posts_data = []
    seen_posts = set()

    # Vorhandene Datei laden, um Duplikate zu vermeiden
    if os.path.exists("factbase_posts_with_ocr.csv"):
        existing_df = pd.read_csv("factbase_posts_with_ocr.csv", encoding="utf-8")
        for _, row in existing_df.iterrows():
            key = (row["author"], row["date"], row["time"], row["text"])
            seen_posts.add(key)
        posts_data = existing_df.to_dict("records")

    page_num = 1

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while True:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")
            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            new_posts_found = False

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Schlüssel für Duplikatprüfung
                    key = (data["author"], data["date"], data["time"], data["text"])
                    if key in seen_posts:
                        continue  # Duplikat überspringen

                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    if local_image_path:
                        ocr_text = extract_text_from_image(local_image_path)
                        metrics = parse_metrics(ocr_text)
                    else:
                        metrics = {"retweets": None, "likes": None, "replies": None}

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path,
                        "retweets": metrics["retweets"],
                        "likes": metrics["likes"],
                        "replies": metrics["replies"],
                    })

                    seen_posts.add(key)
                    new_posts_found = True

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            if not new_posts_found:
                print("Keine neuen Posts auf dieser Seite. Scraping beendet.")
                break

            page_num += 1

        await browser.close()

    # IDs neu vergeben
    posts_data = sorted(posts_data, key=lambda x: (x["year"], x["month"], x["day"], x["time"]))
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_with_ocr.csv", index=False, encoding="utf-8")
    print(f"{len(df)} eindeutige Posts gespeichert.")

In [ ]:
#git clone https://github.com/madmaze/pytesseract.git
#cd pytesseract && pip install -U .

In [ ]:
## Bilder sind jetzt nicht mehr lokal gespeichert, sondern

In [8]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import base64

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

nest_asyncio.apply()  # Für Jupyter-Umgebungen

# Ordner für Bilder anlegen, falls nicht da (optional, falls du mal lokal speichern willst)
os.makedirs("images", exist_ok=True)

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()

        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def download_image_as_base64(session, url):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                img_bytes = await resp.read()
                encoded = base64.b64encode(img_bytes).decode('utf-8')
                return encoded
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 4

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Bild-URL suchen
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    # Bild als Base64 laden
                    image_base64 = None
                    if image_url:
                        image_base64 = await download_image_as_base64(session, image_url)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_data": image_base64  
                    })

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

    # Liste umkehren, damit ältester Post id=1 bekommt
    posts_data.reverse()

    # IDs vergeben
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright_base64.csv", index=False, encoding="utf-8")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright_base64.csv'.")

# Wenn du in Jupyter oder IPython bist, kannst du so starten:
await scrape_factbase()

# Sonst in normalem Skript:
#if __name__ == "__main__":
#    asyncio.run(scrape_factbase())


Scraping von Seite 1
Scraping von Seite 2
Scraping von Seite 3
Scraping von Seite 4
206 Posts erfolgreich gespeichert in 'factbase_posts_playwright_base64.csv'.


In [17]:
import pandas as pd
fb = pd.read_csv("factbase_posts_playwright_base64.csv")
print(fb["image_data"].iloc[15])

...


### Code, der keine Duplikate speichert; plus Bilder, mit asynchronen Workers zum Herunterladen der Bilder: (plus Key)

In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

#sslcontext = ssl.create_default_context(cafile=certifi.where())
nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context()
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "trump_playwright_final.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    os.makedirs("images", exist_ok=True)  # einmalig vor der Schleife
    while True:
        item = await queue.get()
        if item is None:  # Stop-Signal
            queue.task_done()
            break

        image_url, post, filename = item #statt post_idx
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    #with open(fpath, "wb") as f:
                        #f.write(await resp.read())
                    #posts_data[post_idx]["image_path"] = fpath
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

            
def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Content-Zeilen finden
        if len(lines) > 2 and lines[2].startswith("View"): # statt "View on"
            content_lines = lines[3:]
        else:
            content_lines = lines[2:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }
# -------- Duplikat-Key nur aus handle, date, time --------
def make_post_key(data, image_path=None, include_text=False):
    # Handle/Author
    author_val = data.get("author", "")
    if pd.isna(author_val):
        author_val = ""
    author_norm = str(author_val).strip().lower()

    # Datum
    date_val = data.get("date", "")
    if pd.isna(date_val):
        date_val = ""
    date_norm = str(date_val).strip()
    
    #Plattform
    platform_val = data.get("platform", "")
    platform_norm = str(platform_val or "").strip().lower()

    # Zeit
    time_val = data.get("time", "")
    if pd.isna(time_val):
        time_val = ""
    time_norm = str(time_val).strip()
    
    # Text
    text_norm = ""
    if include_text:
        text_val = data.get("text", "")
        if pd.isna(text_val):
            text_val = ""
        text_norm = re.sub(r"\s+", " ", str(text_val).strip().lower())
    
    # Bild
    #image_norm = ""
    #if image_path:
    #    image_norm = str(image_path).strip().lower()
    #image_norm = str(image_path).strip().lower() if image_path else "" #neu
    img_val = data.get("image_path", "")
    if pd.isna(img_val):
        img_val = ""
    img_norm = str(img_val).strip().lower()

    raw_key = f"{author_norm}|{date_norm}|{time_norm}|{img_norm}|{text_norm}" # ergänzen
    return hashlib.sha256(raw_key.encode("utf-8")).hexdigest()



async def scrape_all_dynamic_old(max_posts=90000, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzungslogik ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE)
        posts_data = df_existing.to_dict("records")
        # Korrekte Hash-Keys für alle bereits gespeicherten Posts
        seen_posts = {make_post_key(row) for row in posts_data}
        #seen_posts = {make_post_key(row, row.get("image_path")) for row in posts_data}
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")
 
    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)
        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, posts_data, session)) for _ in range(num_workers)]
        #processed_blocks = 0  # Anzahl der bereits verarbeiteten Blöcke

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                all_texts = await page.eval_on_selector_all(
                    "div.block",
                    "els => els.map(e => e.innerText)"
                )
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            # Alle sichtbaren Blöcke holen
            blocks = await page.query_selector_all("div.block")
            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")
            #new_blocks = all_blocks[processed_blocks:]  # nur neue Blöcke
            #print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(new_blocks)} neue Blöcke gefunden")

            new_count = 0
            for idx, block in enumerate(blocks): #new_blocks
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)


                    # --- Bild-Download ---
                    local_image_path = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                #local_image_path = os.path.join("images", filename) #der Teil ist neu
                                await queue.put((src, len(posts_data), filename))
                    except Exception as e: #vorher stand hier nur pass (und Bild und Key Teile waren getauscht)
                        print(f"Fehler beim Verarbeiten des Bildes: {e}")
                    
                    # --- Eindeutigen Schlüssel bauen ---
                    key = make_post_key(post, include_text=True)
                    #key = make_post_key({
                    #    "author": data["author"],
                    #    "date": data["date"],
                    #    "time": data["time"],
                    #    "text": data["text"],
                    #    "image_path": data.get("image_path")
                    #}, include_text=True)
                    #key = f"{data['author']}|{data['date']}|{data['time']}|{data['text'] or ''}"
                    if key in seen_posts:
                        continue
                    seen_posts.add(key)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path  # wird später im Download-Worker gesetzt
                    })

                    new_count += 1

                    # Abbruchbedingung
                    if len(posts_data) >= max_posts:
                        print(f"Maximale Anzahl {max_posts} erreicht.")
                        no_new_rounds = max_no_new
                        break

                except Exception as e:
                    print(f"Fehler bei Post: {e}")
            
            #processed_blocks = len(all_blocks)  # Update der bereits verarbeiteten Blöcke
            print(f"Neu hinzugekommen: {new_count} Posts")

            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen
            last_height = await page.evaluate("document.body.scrollHeight")
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(1)
            new_height = await page.evaluate("document.body.scrollHeight")

            if new_height == last_height:
                print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                break

        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)
        await browser.close()

    # IDs vergeben
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

async def scrape_all_dynamic(max_posts=90000, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzung ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE)
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data}
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None
                    }

                    # Schlüssel generieren (erstmal ohne Bild)
                    key = make_post_key(post)
                    if key in seen_posts:
                        continue
                    seen_posts.add(key)

                    # Bild-Download vorbereiten
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                await queue.put((src, post, filename))
                    except:
                        pass

                    posts_data.append(post)
                    new_count += 1

                    if len(posts_data) >= max_posts:
                        print(f"Maximale Anzahl {max_posts} erreicht.")
                        no_new_rounds = max_no_new
                        break

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen
            last_height = await page.evaluate("document.body.scrollHeight")
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(3)
            new_height = await page.evaluate("document.body.scrollHeight")

            if new_height == last_height:
                print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx


    fff = pd.DataFrame(posts_data)# .fillna("")
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")

# --- Analyse: wie viele Posts haben Text ---
    total = len(fff)
    no_text = fff["text"].isna().sum()
    with_text = total - no_text
    print("===================================")
    print(f"Gesamt:     {total}")
    print(f"Ohne Text:  {no_text} ({no_text/total:.1%})")
    print(f"Mit Text:   {with_text} ({with_text/total:.1%})")
    print("===================================")

# Starten
await scrape_all_dynamic(max_posts=90000)

### keine doppelten Posts und laden, bis keine neuen Posts gefunden werden

In [26]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl

sslcontext = ssl.create_default_context(cafile=certifi.where())

nest_asyncio.apply()  # Für Jupyter-Umgebungen

os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context()
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()

        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def scrape_factbase():
    posts_data = []
    seen_posts = set()
    page_num = 1

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while True:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")
            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            new_posts_found = False

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    key = (data["author"], data["date"], data["time"], data["text"])
                    if key in seen_posts:
                        continue  # Duplikat überspringen

                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path
                    })

                    seen_posts.add(key)
                    new_posts_found = True

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            if not new_posts_found:
                print("Keine neuen Posts auf dieser Seite. Scraping beendet.")
                break

            page_num += 1

        await browser.close()

    posts_data.reverse()

    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright_full.csv", index=False, encoding="utf-8")
    print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright_full.csv'.")

# Für Notebook:
await scrape_factbase()


Scraping von Seite 1
Scraping von Seite 2
Keine neuen Posts auf dieser Seite. Scraping beendet.
51 Posts erfolgreich gespeichert in 'factbase_posts_playwright_full.csv'.


### mit Scrollfunktion

In [6]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import time

nest_asyncio.apply()

sslcontext = ssl.create_default_context(cafile=certifi.where())

os.makedirs("images", exist_ok=True)

sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

def extract_metadata_text(text: str):
    # Deine Funktion unverändert hier rein
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()

        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def scrape_all_dynamic():
    posts_data = []
    unique_texts = set()

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page=1")

        more_button_selector = "button.load-more"  # Beispiel, ggf. anpassen!

        consecutive_no_new_posts = 0
        max_no_new = 5

        while True:
            await page.wait_for_selector("div.block", timeout=10000)

            blocks = await page.query_selector_all("div.block")
            print(f"Aktuell {len(blocks)} Posts gespeichert")

            new_posts_count = 0
            for block in blocks:
                full_text = await block.inner_text()
                data = extract_metadata_text(full_text)

                if data["text"] not in unique_texts:
                    unique_texts.add(data["text"])

                    # Bild-URL suchen
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path
                    })
                    new_posts_count += 1

            print(f"{new_posts_count} neue Posts gefunden")

            if new_posts_count == 0:
                consecutive_no_new_posts += 1
                print(f"Keine neuen Posts in dieser Runde ({consecutive_no_new_posts} mal hintereinander)")
                if consecutive_no_new_posts >= max_no_new:
                    print("Maximale Anzahl leerer Durchläufe erreicht. Stoppe Scraping.")
                    break
            else:
                consecutive_no_new_posts = 0

            # Wenn ein „Mehr laden“-Button da ist, klicken
            try:
                more_button = await page.query_selector(more_button_selector)
                if more_button:
                    is_disabled = await more_button.get_attribute("disabled")
                    if not is_disabled:
                        print("Klicke 'Mehr laden' Button")
                        await more_button.click()
                        await asyncio.sleep(3)  # Warte auf Nachladen
                        continue
            except Exception as e:
                print(f"Fehler beim Klick auf 'Mehr laden': {e}")

            # Alternativ scrollen, falls kein Button
            print("Scrollen zum Nachladen")
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(3)

        await browser.close()

    # IDs vergeben & CSV speichern
    posts_data.reverse()
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright_full_dynamic.csv", index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamtzahl der Posts: {len(df)}. Datei gespeichert als 'factbase_posts_playwright_full_dynamic.csv'.")

# Im Jupyter Notebook oder Async-fähiger Umgebung:
await scrape_all_dynamic()


Aktuell 51 Posts im DOM
43 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 101 Posts im DOM
40 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 151 Posts im DOM
44 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 201 Posts im DOM
43 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 251 Posts im DOM
30 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 301 Posts im DOM
42 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 351 Posts im DOM
40 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 401 Posts im DOM
25 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 451 Posts im DOM
41 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 501 Posts im DOM
34 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 551 Posts im DOM
39 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 601 Posts im DOM
41 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 651 Posts im DOM
38 neue Posts gefunden
Scrollen zum Nachladen
Aktuell 701 Posts im DOM
32 neue Posts gefunden
Scrollen zum Nachladen
Aktuell

TimeoutError: Page.wait_for_selector: Timeout 10000ms exceeded.
Call log:
  - waiting for locator("div.block") to be visible


In [ ]:
## mit Seitenanzzahl, aber ohne doppelte Posts

In [ ]:
import hashlib
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl

sslcontext = ssl.create_default_context(cafile=certifi.where())

nest_asyncio.apply()  # Für Jupyter-Umgebungen

# Ordner für Bilder anlegen, falls nicht da
os.makedirs("images", exist_ok=True)


sslcontext = ssl.create_default_context()
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None


def extract_metadata_text(text: str):
    # ... (Deine bestehende Funktion, unverändert)
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()

        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )

        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 40000

    seen_posts = set()  # Für aktuelle Sitzung: um doppelte Posts zu erkennen

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(8000000)

        while page_num <= max_pages:
            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url)
                await page.wait_for_selector("div.block", timeout=800000)
            except Exception as e:
                print(f"Fehler beim Laden von Seite {page_num}: {e}")
                break

            blocks = await page.query_selector_all("div.block")

            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Schlüssel für Duplikatprüfung
                    key_str = f"{data['author']}|{data['date']}|{data['time']}|{data['text']}"
                    key_hash = hashlib.sha256(key_str.encode('utf-8')).hexdigest()

                    if key_hash in seen_posts:
                        # Duplikat gefunden, überspringen
                        continue

                    # Bild-URL suchen
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    # Bild herunterladen und lokal speichern
                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    posts_data.append({
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": local_image_path
                    })

                    seen_posts.add(key_hash)  # Jetzt Post merken

                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            page_num += 1

        await browser.close()

        posts_data.reverse()
        for idx, post in enumerate(posts_data, start=1):
            post["id"] = idx

        df = pd.DataFrame(posts_data)
        df.to_csv("factbase_posts_playwright15.csv", index=False, encoding="utf-8")
        print(f"{len(df)} Posts erfolgreich gespeichert in 'factbase_posts_playwright15.csv'.")

await scrape_factbase()

Scraping von Seite 1
Scraping von Seite 2
Scraping von Seite 3
Scraping von Seite 4
Scraping von Seite 5
Scraping von Seite 6
Scraping von Seite 7
Scraping von Seite 8
Scraping von Seite 9
Scraping von Seite 10
Scraping von Seite 11
Scraping von Seite 12
Scraping von Seite 13
Scraping von Seite 14
Scraping von Seite 15
Scraping von Seite 16
Scraping von Seite 17
Scraping von Seite 18
Scraping von Seite 19
Scraping von Seite 20
Scraping von Seite 21
Scraping von Seite 22
Scraping von Seite 23
Scraping von Seite 24
Scraping von Seite 25
Scraping von Seite 26
Scraping von Seite 27
Scraping von Seite 28
Scraping von Seite 29
Scraping von Seite 30
Scraping von Seite 31
Scraping von Seite 32
Scraping von Seite 33
Scraping von Seite 34
Scraping von Seite 35
Scraping von Seite 36
Scraping von Seite 37
Scraping von Seite 38
Scraping von Seite 39
Scraping von Seite 40
Scraping von Seite 41
Scraping von Seite 42
Scraping von Seite 43
Scraping von Seite 44
Scraping von Seite 45
Scraping von Seite 

Scraping von Seite 362
Scraping von Seite 363
Scraping von Seite 364
Scraping von Seite 365
Scraping von Seite 366
Scraping von Seite 367
Scraping von Seite 368
Scraping von Seite 369
Scraping von Seite 370
Scraping von Seite 371
Scraping von Seite 372
Scraping von Seite 373
Scraping von Seite 374
Scraping von Seite 375
Scraping von Seite 376
Scraping von Seite 377
Scraping von Seite 378
Scraping von Seite 379
Scraping von Seite 380
Scraping von Seite 381
Scraping von Seite 382
Scraping von Seite 383
Scraping von Seite 384
Scraping von Seite 385
Scraping von Seite 386
Scraping von Seite 387
Scraping von Seite 388
Scraping von Seite 389
Scraping von Seite 390
Scraping von Seite 391
Scraping von Seite 392
Scraping von Seite 393
Scraping von Seite 394
Scraping von Seite 395
Scraping von Seite 396
Scraping von Seite 397
Scraping von Seite 398
Scraping von Seite 399
Scraping von Seite 400
Scraping von Seite 401
Scraping von Seite 402
Scraping von Seite 403
Scraping von Seite 404
Scraping vo

In [4]:
import pandas as pd
fs = pd.read_csv("factbase_posts_playwright15.csv")
print(fs.head(10))

                          author      platform        date   time   day  \
0  Donald Trump @realDonaldTrump  Truth Social  2025-08-12  12:08  12.0   
1  Donald Trump @realDonaldTrump  Truth Social  2025-08-12  15:30  12.0   
2  Donald Trump @realDonaldTrump  Truth Social  2025-08-12  15:33  12.0   
3  Donald Trump @realDonaldTrump  Truth Social  2025-08-12  15:35  12.0   
4                            NaN           NaN         NaN    NaN   NaN   
5         THE SOURCE FOR NEWS ON           NaN         NaN    NaN   NaN   
6  Donald Trump @realDonaldTrump  Truth Social  2025-08-09  08:56   9.0   
7  Donald Trump @realDonaldTrump  Truth Social  2025-08-09  09:02   9.0   
8  Donald Trump @realDonaldTrump  Truth Social  2025-08-09  09:04   9.0   
9  Donald Trump @realDonaldTrump  Truth Social  2025-08-09  09:08   9.0   

    month    year                                               text  \
0  August  2025.0  RT: https://truthsocial.com/users/realDonaldTr...   
1  August  2025.0  Jeanine Pir

In [5]:
fs.shape

(56, 10)

In [ ]:
## der Code lädt bis in die Ewigkeit!
import hashlib
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl

# SSL-Setup
sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

nest_asyncio.apply()

# Bilder-Ordner anlegen
os.makedirs("images", exist_ok=True)

# --------------------
# Hilfsfunktionen
# --------------------
async def download_image(session, url, filename):
    try:
        async with session.get(url, ssl=sslcontext, timeout=30) as resp:
            if resp.status == 200:
                fpath = os.path.join("images", filename)
                with open(fpath, "wb") as f:
                    f.write(await resp.read())
                return fpath
    except Exception as e:
        print(f"Fehler beim Download des Bildes {url}: {e}")
    return None

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 3:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()
        content_lines = lines[3:]

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": "\n".join(content_lines).strip()
        }
    except:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": "\n".join(content_lines).strip()
        }

# --------------------
# Hauptfunktion
# --------------------
async def scrape_factbase():
    posts_data = []
    page_num = 1
    max_pages = 9000  # hoch genug, reale Begrenzung kommt durch Abbruchbedingungen

    target_post_count = 90000
    no_new_posts_limit = 88000  # Seiten ohne neue Posts bis Abbruch
    no_new_posts_counter = 0

    seen_posts = set()

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(30000)  # 30 Sekunden pro Seite

        while page_num <= max_pages:
            if len(posts_data) >= target_post_count:
                print(f"Ziel von {target_post_count} Posts erreicht.")
                break

            url = f"https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc&page={page_num}"
            print(f"Scraping von Seite {page_num}")

            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_selector("div.block", timeout=15000)
            except Exception as e:
                print(f"⚠️ Seite {page_num} übersprungen (Fehler/Timeout): {e}")
                page_num += 1
                continue

            blocks = await page.query_selector_all("div.block")
            if not blocks:
                print("Keine weiteren Posts gefunden. Scraping beendet.")
                break

            new_posts_this_page = 0

            for block in blocks:
                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Duplikat-Prüfung
                    key_str = f"{data['author']}|{data['date']}|{data['time']}|{data['text']}"
                    key_hash = hashlib.sha256(key_str.encode('utf-8')).hexdigest()
                    if key_hash in seen_posts:
                        continue

                    # Bild verarbeiten
                    image_url = None
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                image_url = src
                    except:
                        pass

                    local_image_path = None
                    if image_url:
                        filename = image_url.split("/")[-1].split("?")[0]
                        filename = f"{data['date']}_{data['time'].replace(':','')}_{filename}"
                        local_image_path = await download_image(session, image_url, filename)

                    data["image_path"] = local_image_path
                    posts_data.append(data)
                    seen_posts.add(key_hash)
                    new_posts_this_page += 1

                    if len(posts_data) >= target_post_count:
                        break
                except Exception as e:
                    print(f"Fehler bei Post auf Seite {page_num}: {e}")
                    continue

            if new_posts_this_page == 0:
                no_new_posts_counter += 1
            else:
                no_new_posts_counter = 0

            if no_new_posts_counter >= no_new_posts_limit:
                print(f"Abbruch nach {no_new_posts_counter} Seiten ohne neue Posts.")
                break

            page_num += 1

        await browser.close()

    # Sortieren & ID vergeben
    posts_data.reverse()
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    df = pd.DataFrame(posts_data)
    df.to_csv("factbase_posts_playwright18.csv", index=False, encoding="utf-8")
    print(f"{len(df)} eindeutige Posts erfolgreich gespeichert in 'factbase_posts_playwright18.csv'.")

# --------------------
# Start
# --------------------
await scrape_factbase()


Scraping von Seite 1
Scraping von Seite 2
Scraping von Seite 3
Scraping von Seite 4
Scraping von Seite 5
Scraping von Seite 6
Scraping von Seite 7
Scraping von Seite 8
Scraping von Seite 9
Scraping von Seite 10
Scraping von Seite 11
Scraping von Seite 12
Scraping von Seite 13
Scraping von Seite 14
Scraping von Seite 15
Scraping von Seite 16
Scraping von Seite 17
Scraping von Seite 18
Scraping von Seite 19
Scraping von Seite 20
Scraping von Seite 21
Scraping von Seite 22
Scraping von Seite 23
Scraping von Seite 24
Scraping von Seite 25
Scraping von Seite 26
Scraping von Seite 27
Scraping von Seite 28
Scraping von Seite 29
Scraping von Seite 30
Scraping von Seite 31
Scraping von Seite 32
Scraping von Seite 33
Scraping von Seite 34
Scraping von Seite 35
Scraping von Seite 36
Scraping von Seite 37
Scraping von Seite 38
Scraping von Seite 39
Scraping von Seite 40
Scraping von Seite 41
Scraping von Seite 42
Scraping von Seite 43
Scraping von Seite 44
Scraping von Seite 45
Scraping von Seite 

Scraping von Seite 362
Scraping von Seite 363
Scraping von Seite 364
Scraping von Seite 365
Scraping von Seite 366
Scraping von Seite 367
Scraping von Seite 368
Scraping von Seite 369
Scraping von Seite 370
Scraping von Seite 371
Scraping von Seite 372
Scraping von Seite 373
Scraping von Seite 374
Scraping von Seite 375
Scraping von Seite 376
Scraping von Seite 377
Scraping von Seite 378
Scraping von Seite 379
Scraping von Seite 380
Scraping von Seite 381
Scraping von Seite 382
Scraping von Seite 383
Scraping von Seite 384
Scraping von Seite 385
Scraping von Seite 386
Scraping von Seite 387
Scraping von Seite 388
Scraping von Seite 389
Scraping von Seite 390
Scraping von Seite 391
Scraping von Seite 392
Scraping von Seite 393
Scraping von Seite 394
Scraping von Seite 395
Scraping von Seite 396
Scraping von Seite 397
Scraping von Seite 398
Scraping von Seite 399
Scraping von Seite 400
Scraping von Seite 401
Scraping von Seite 402
Scraping von Seite 403
Scraping von Seite 404
Scraping vo

Scraping von Seite 719
Scraping von Seite 720
Scraping von Seite 721
Scraping von Seite 722
Scraping von Seite 723
Scraping von Seite 724
Scraping von Seite 725
Scraping von Seite 726
Scraping von Seite 727
Scraping von Seite 728
Scraping von Seite 729
Scraping von Seite 730
Scraping von Seite 731
Scraping von Seite 732
Scraping von Seite 733
Scraping von Seite 734
Scraping von Seite 735
Scraping von Seite 736
Scraping von Seite 737
Scraping von Seite 738
Scraping von Seite 739
Scraping von Seite 740
Scraping von Seite 741
Scraping von Seite 742
Scraping von Seite 743
Scraping von Seite 744
Scraping von Seite 745
Scraping von Seite 746
Scraping von Seite 747
Scraping von Seite 748
Scraping von Seite 749
Scraping von Seite 750
Scraping von Seite 751
Scraping von Seite 752
Scraping von Seite 753
Scraping von Seite 754
Scraping von Seite 755
Scraping von Seite 756
Scraping von Seite 757
Scraping von Seite 758
Scraping von Seite 759
Scraping von Seite 760
Scraping von Seite 761
Scraping vo

Scraping von Seite 1073
Scraping von Seite 1074
Scraping von Seite 1075
Scraping von Seite 1076
Scraping von Seite 1077
Scraping von Seite 1078
Scraping von Seite 1079
Scraping von Seite 1080
Scraping von Seite 1081
Scraping von Seite 1082
Scraping von Seite 1083
Scraping von Seite 1084
Scraping von Seite 1085
Scraping von Seite 1086
Scraping von Seite 1087
Scraping von Seite 1088
Scraping von Seite 1089
Scraping von Seite 1090
Scraping von Seite 1091
Scraping von Seite 1092
Scraping von Seite 1093
Scraping von Seite 1094
Scraping von Seite 1095
Scraping von Seite 1096
Scraping von Seite 1097
Scraping von Seite 1098
Scraping von Seite 1099
Scraping von Seite 1100
Scraping von Seite 1101
Scraping von Seite 1102
Scraping von Seite 1103
Scraping von Seite 1104
Scraping von Seite 1105
Scraping von Seite 1106
Scraping von Seite 1107
Scraping von Seite 1108
Scraping von Seite 1109
Scraping von Seite 1110
Scraping von Seite 1111
Scraping von Seite 1112
Scraping von Seite 1113
Scraping von Sei

Scraping von Seite 1415
Scraping von Seite 1416
Scraping von Seite 1417
Scraping von Seite 1418
Scraping von Seite 1419
Scraping von Seite 1420
Scraping von Seite 1421
Scraping von Seite 1422
Scraping von Seite 1423
Scraping von Seite 1424
Scraping von Seite 1425
Scraping von Seite 1426
Scraping von Seite 1427
Scraping von Seite 1428
Scraping von Seite 1429
Scraping von Seite 1430
Scraping von Seite 1431
Scraping von Seite 1432
Scraping von Seite 1433
Scraping von Seite 1434
Scraping von Seite 1435
Scraping von Seite 1436
Scraping von Seite 1437
Scraping von Seite 1438
Scraping von Seite 1439
Scraping von Seite 1440
Scraping von Seite 1441
Scraping von Seite 1442
Scraping von Seite 1443
Scraping von Seite 1444
Scraping von Seite 1445
Scraping von Seite 1446
Scraping von Seite 1447
Scraping von Seite 1448
Scraping von Seite 1449
Scraping von Seite 1450
Scraping von Seite 1451
Scraping von Seite 1452
Scraping von Seite 1453
Scraping von Seite 1454
Scraping von Seite 1455
Scraping von Sei

...

Scraping von Seite 10604
Scraping von Seite 10605
Scraping von Seite 10606
Scraping von Seite 10607
Scraping von Seite 10608
Scraping von Seite 10609
Scraping von Seite 10610
Scraping von Seite 10611
Scraping von Seite 10612
Scraping von Seite 10613
Scraping von Seite 10614
Scraping von Seite 10615
Scraping von Seite 10616
Scraping von Seite 10617
Scraping von Seite 10618
Scraping von Seite 10619
Scraping von Seite 10620
Scraping von Seite 10621
Scraping von Seite 10622
Scraping von Seite 10623
Scraping von Seite 10624
Scraping von Seite 10625
Scraping von Seite 10626
Scraping von Seite 10627
Scraping von Seite 10628
Scraping von Seite 10629
Scraping von Seite 10630
Scraping von Seite 10631
Scraping von Seite 10632
Scraping von Seite 10633
Scraping von Seite 10634
Scraping von Seite 10635
Scraping von Seite 10636
Scraping von Seite 10637
Scraping von Seite 10638
Scraping von Seite 10639
Scraping von Seite 10640
Scraping von Seite 10641
Scraping von Seite 10642
Scraping von Seite 10643


Scraping von Seite 10932
Scraping von Seite 10933
Scraping von Seite 10934
Scraping von Seite 10935
Scraping von Seite 10936
Scraping von Seite 10937
Scraping von Seite 10938
Scraping von Seite 10939
Scraping von Seite 10940
Scraping von Seite 10941
Scraping von Seite 10942
Scraping von Seite 10943
Scraping von Seite 10944
Scraping von Seite 10945
Scraping von Seite 10946
Scraping von Seite 10947
Scraping von Seite 10948
Scraping von Seite 10949
Scraping von Seite 10950
Scraping von Seite 10951
Scraping von Seite 10952
Scraping von Seite 10953
Scraping von Seite 10954
Scraping von Seite 10955
Scraping von Seite 10956
Scraping von Seite 10957
Scraping von Seite 10958
Scraping von Seite 10959
Scraping von Seite 10960
Scraping von Seite 10961
Scraping von Seite 10962
Scraping von Seite 10963
Scraping von Seite 10964
Scraping von Seite 10965
Scraping von Seite 10966
Scraping von Seite 10967
Scraping von Seite 10968
Scraping von Seite 10969
Scraping von Seite 10970
Scraping von Seite 10971


Scraping von Seite 11260
Scraping von Seite 11261
Scraping von Seite 11262
Scraping von Seite 11263
Scraping von Seite 11264
Scraping von Seite 11265
Scraping von Seite 11266
Scraping von Seite 11267
Scraping von Seite 11268
Scraping von Seite 11269
Scraping von Seite 11270
Scraping von Seite 11271
Scraping von Seite 11272
Scraping von Seite 11273
Scraping von Seite 11274
Scraping von Seite 11275
Scraping von Seite 11276
Scraping von Seite 11277
Scraping von Seite 11278
Scraping von Seite 11279
Scraping von Seite 11280
Scraping von Seite 11281
Scraping von Seite 11282
Scraping von Seite 11283
Scraping von Seite 11284
Scraping von Seite 11285
Scraping von Seite 11286
Scraping von Seite 11287
Scraping von Seite 11288
Scraping von Seite 11289
Scraping von Seite 11290
Scraping von Seite 11291
Scraping von Seite 11292
Scraping von Seite 11293
Scraping von Seite 11294
Scraping von Seite 11295
Scraping von Seite 11296
Scraping von Seite 11297
Scraping von Seite 11298
Scraping von Seite 11299


Scraping von Seite 11588
Scraping von Seite 11589
Scraping von Seite 11590
Scraping von Seite 11591
Scraping von Seite 11592
Scraping von Seite 11593
Scraping von Seite 11594
Scraping von Seite 11595
Scraping von Seite 11596
Scraping von Seite 11597
Scraping von Seite 11598
Scraping von Seite 11599
Scraping von Seite 11600
Scraping von Seite 11601
Scraping von Seite 11602
Scraping von Seite 11603
Scraping von Seite 11604
Scraping von Seite 11605
Scraping von Seite 11606
Scraping von Seite 11607
Scraping von Seite 11608
Scraping von Seite 11609
Scraping von Seite 11610
Scraping von Seite 11611
Scraping von Seite 11612
Scraping von Seite 11613
Scraping von Seite 11614
Scraping von Seite 11615
Scraping von Seite 11616
Scraping von Seite 11617
Scraping von Seite 11618
Scraping von Seite 11619
Scraping von Seite 11620
Scraping von Seite 11621
Scraping von Seite 11622
Scraping von Seite 11623
Scraping von Seite 11624
Scraping von Seite 11625
Scraping von Seite 11626
Scraping von Seite 11627


Scraping von Seite 11916
Scraping von Seite 11917
Scraping von Seite 11918
Scraping von Seite 11919
Scraping von Seite 11920
Scraping von Seite 11921
Scraping von Seite 11922
Scraping von Seite 11923
Scraping von Seite 11924
Scraping von Seite 11925
Scraping von Seite 11926
Scraping von Seite 11927
Scraping von Seite 11928
Scraping von Seite 11929
Scraping von Seite 11930
Scraping von Seite 11931
Scraping von Seite 11932
Scraping von Seite 11933
Scraping von Seite 11934
Scraping von Seite 11935
Scraping von Seite 11936
Scraping von Seite 11937
Scraping von Seite 11938
Scraping von Seite 11939
Scraping von Seite 11940
Scraping von Seite 11941
Scraping von Seite 11942
Scraping von Seite 11943
Scraping von Seite 11944
Scraping von Seite 11945
Scraping von Seite 11946
Scraping von Seite 11947
Scraping von Seite 11948
Scraping von Seite 11949
Scraping von Seite 11950
Scraping von Seite 11951
Scraping von Seite 11952
Scraping von Seite 11953
Scraping von Seite 11954
Scraping von Seite 11955
